In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
from google.cloud import bigquery
import time
import calendar
from datetime import datetime, timedelta,date
import seaborn as sns
import pickle
import cloudpickle
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
client = bigquery.Client()
from tqdm import tqdm
import logging
import sys
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
client = bigquery.Client()
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import warnings as w
w.filterwarnings('ignore')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from pyspark.sql import types as st
from pyspark.sql.window import Window

In [3]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName('SMS Feature Mart') \
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.29.0') \
    .config('spark.jars', '/home/jupyter/jar_folder/gcs-connector-hadoop3-latest.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .config('spark.hadoop.google.cloud.auth.service.account.enable', 'true')\
    .config('spark.executor.instances', '8') \
    .config('spark.executor.memory', '64g') \
    .config('spark.driver.memory', '16g') \
    .config('spark.memory.fraction', '0.8') \
    .config('spark.sql.shuffle.partitions', '1000') \
    .config('spark.shuffle.spill', 'true') \
    .config('spark.hadoop.fs.gs.system.bucket', 'alternate_credit_score_model_files') \
    .getOrCreate()

# Step 2: Set up GCS configurations
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("google.cloud.auth.service.account.enable", "true")
hadoop_conf.set("google.cloud.auth.service.account.json.keyfile", "/home/jupyter/jar_folder/abcd-dataplatform-4a30abb96536.json")
print("Spark Session created")

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5d61e7d8-e722-433e-8e5f-e13fa21900ec;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.29.0 in central
:: resolution report :: resolve 202ms :: artifacts dl 3ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.29.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------

25/06/06 12:45:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Session created


In [4]:
#creating date variable 
target_month = pd.to_datetime('2025-04-01')  # Always Remember first of the month(target month starting onward march...oct)

# Feature end date is 1 day before target month
feature_end_date = target_month - pd.DateOffset(days=1)

# Define time windows
windows = [1, 3, 6, 12]
date_windows = {}

for window in windows:
    start_date = feature_end_date - pd.DateOffset(months=window-1)
    start_date = start_date.replace(day=1)  # Make it first day of that month
    date_windows[f'last_{window}_months'] = {
        'start_date': start_date.strftime('%Y-%m-%d'),
        'end_date': feature_end_date.strftime('%Y-%m-%d')
    }

# Print date windows
for window, dates in date_windows.items():
    print(f"{window}: {dates}")

last_1_months: {'start_date': '2025-03-01', 'end_date': '2025-03-31'}
last_3_months: {'start_date': '2025-01-01', 'end_date': '2025-03-31'}
last_6_months: {'start_date': '2024-10-01', 'end_date': '2025-03-31'}
last_12_months: {'start_date': '2024-04-01', 'end_date': '2025-03-31'}


In [5]:
# #defining date variable
# last_3months = as_of_end_dt - pd.DateOffset(months=3)
# last_6months = as_of_end_dt - pd.DateOffset(months=6)
# last_9months = as_of_end_dt - pd.DateOffset(months=9)
# last_12months = as_of_end_dt - pd.DateOffset(months=12)

In [6]:
created_date_cutoff = pd.to_datetime(feature_end_date).normalize()
created_date_cutoff

Timestamp('2025-03-31 00:00:00')

# Model Base

In [7]:
model_base_query = f"""
    SELECT
    mobilenumber,
    id AS customer_id
FROM 
    `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer`
WHERE
    DATE(createddate) <= DATE('{created_date_cutoff}')
        """
model_base = client.query(model_base_query).to_dataframe()
model_base['mobilenumber'] = pd.to_numeric(model_base['mobilenumber'], errors='coerce')
model_base = model_base.dropna(subset=['mobilenumber'])
model_base['mobilenumber'] = model_base['mobilenumber'].astype('int64')
model_base = model_base[model_base['mobilenumber'].between(10**9, 10**10 - 1)]
model_base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5077450 entries, 0 to 5077531
Data columns (total 2 columns):
 #   Column        Dtype
---  ------        -----
 0   mobilenumber  int64
 1   customer_id   Int64
dtypes: Int64(1), int64(1)
memory usage: 121.1 MB


In [8]:
print(model_base_query)


    SELECT
    mobilenumber,
    id AS customer_id
FROM 
    `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer`
WHERE
    DATE(createddate) <= DATE('2025-03-31 00:00:00')
        


# Target Base snapshot month april

In [9]:
model_base.shape

(5077450, 2)

# creating feature from sms_date timebased 

# Extracting Investment related feature for last 1yrs

In [10]:
# Set Google Cloud project ID
project_id = 'abcd-dataplatform-prod'
dataset_id = "abcd_digitap_transformed"
table_id = "users_digitap_sms_data_transformed"
table_name = f"{project_id}.{dataset_id}.{table_id}"

sms_data = spark.read.format("bigquery") \
    .option('table', table_name) \
    .load()

In [11]:
sms_data.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- year_month: string (nullable = true)
 |-- year_month_date: date (nullable = true)
 |-- total_inflow: string (nullable = true)
 |-- total_expense: string (nullable = true)
 |-- balances: struct (nullable = true)
 |    |-- bank_accounts: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- balance: string (nullable = true)
 |    |    |    |-- monthly_credit: string (nullable = true)
 |    |    |    |-- monthly_debit: string (nullable = true)
 |    |    |    |-- bank: string (nullable = true)
 |    |    |    |-- account: string (nullable = true)
 |    |    |    |-- last_txn_date: string (nullable = true)
 |    |-- year_month: string (nullable = true)
 |    |-- total_inflow: string (nullable = true)
 |    |-- loan_dues: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- emi_due: string (nullable = true)
 |    |  

In [12]:
test_columns = ['user_id', 'ingestion_time', 'device_id', 'year_month', 'year_month_date']
print(''.join([f"    sf.col('{col}'),\n" for col in test_columns]))

    sf.col('user_id'),
    sf.col('ingestion_time'),
    sf.col('device_id'),
    sf.col('year_month'),
    sf.col('year_month_date'),



In [13]:
investment_base = sms_data.select(test_columns + ['balances.investment'])
investment_base.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- ingestion_time: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- year_month: string (nullable = true)
 |-- year_month_date: date (nullable = true)
 |-- investment: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- amount: string (nullable = true)
 |    |    |-- folio_no: string (nullable = true)
 |    |    |-- last_txn_date: string (nullable = true)



In [14]:
# Explode the investment data
investments_df = investment_base.select(
    sf.col("user_id"),
    sf.col('year_month'),
    sf.col("year_month_date"),
    sf.explode(sf.col("investment")).alias("investment"))

In [15]:
# Fetch only the last 1 years of investment data
max_date = investments_df.agg(sf.max("year_month_date")).collect()[0][0]
twelve_months_ago = sf.add_months(sf.lit(max_date), -12)

investments_df = investments_df.filter(
    (sf.col("year_month_date") > twelve_months_ago) & (sf.col("year_month_date") <= sf.lit(max_date))
)

In [16]:
# Select required columns with necessary transformations
investments_df = investments_df.select(
    sf.col("user_id"),
    sf.col('year_month_date'),
    sf.col("investment.type").alias("investment_type"),
    sf.regexp_replace(sf.col("investment.amount"), ',', '').cast("float").alias("investment_amount"),
    sf.col("investment.folio_no"),
    sf.col("investment.last_txn_date")
)

In [17]:
investments_df.select("investment_type").distinct().show(truncate=False)

+-----------------------+
|investment_type        |
+-----------------------+
|SHARE & TRADING        |
|RECURRING DEPOSIT      |
|RD BOOKING             |
|FIXED DEPOSIT          |
|TD BOOKING             |
|RD CLOSURE/ RD MATURE  |
|NPS CONTRIBUTION       |
|NPS INVESTED SUM       |
|TERM DEPOSIT           |
|TD CLOSURE/ TD MATURE  |
|FD BOOKING             |
|FD CLOSURE/ FD MATURE  |
|EMPLOYEE PROVIDENT FUND|
|MF REDEMPTION          |
|MF EXPENSE             |
|Mutual Fund            |
+-----------------------+



In [18]:
#adding new feature to pyspark investment df 
investments_df = investments_df.withColumn("investment_type", sf.lower(sf.col("investment_type")))
investments_df = investments_df.withColumn("last_txn_date", sf.col("last_txn_date").cast(st.DateType()))

In [19]:
investments_df = investments_df.withColumn(
    "grouped_investment_type",
    sf.when(sf.col("investment_type").rlike(r"(?i)(share & trading)"), "trading")
    .when(sf.col("investment_type").rlike(r"(?i)(recurring deposit|rd booking)"), "recurring_deposit")
    .when(sf.col("investment_type").rlike(r"(?i)(fixed deposit|fd booking)"), "fixed_deposit")
    .when(sf.col("investment_type").rlike(r"(?i)(term deposit|td booking)"), "term_deposit")
    .when(sf.col("investment_type").rlike(r"(?i)(employee provident fund)"), "provident_fund")
    .when(sf.col("investment_type").rlike(r"(?i)(nps contribution|nps invested sum)"), "nps")
    .when(sf.col("investment_type").rlike(r"(?i)(mutual fund)"), "mutual_fund")

    .otherwise("Others")
)

In [20]:
as_of_end_dt = feature_end_date
as_of_end_dt

Timestamp('2025-03-31 00:00:00')

In [21]:
date_windows

{'last_1_months': {'start_date': '2025-03-01', 'end_date': '2025-03-31'},
 'last_3_months': {'start_date': '2025-01-01', 'end_date': '2025-03-31'},
 'last_6_months': {'start_date': '2024-10-01', 'end_date': '2025-03-31'},
 'last_12_months': {'start_date': '2024-04-01', 'end_date': '2025-03-31'}}

In [22]:
date_windows['last_1_months']

{'start_date': '2025-03-01', 'end_date': '2025-03-31'}

In [23]:
inv_query = f"""
WITH exploded_investments AS (
  SELECT 
    SAFE_CAST(RIGHT(REPLACE(user_id, ',', ''), 10) AS INT64) AS user_id,
    year_month_date,
    inv.type AS investment_type,
    SAFE_CAST(REPLACE(inv.amount, ',', '') AS FLOAT64) AS investment_amount,
    inv.folio_no,
    PARSE_DATE('%Y-%m-%d', inv.last_txn_date) AS last_txn_date
  FROM 
    `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` AS a,
    UNNEST(balances.investment) AS inv
  WHERE 
    year_month_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 12 MONTH) AND DATE('{as_of_end_dt}')
),

grouped_investments AS (
  SELECT 
    user_id,
    year_month_date,
    investment_amount,
    folio_no,
    last_txn_date,
    CASE 
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(fixed deposit|fd booking|fd)") THEN "fixed_deposit"
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(recurring deposit|rd booking|rd)") THEN "recurring_deposit"
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(term deposit|td booking|td)") THEN "term_deposit"
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(employee provident fund|provident fund)") THEN "provident_fund"
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(nps contribution|nps invested sum|nps)") THEN "nps"
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(mutual fund|mf)") THEN "mutual_fund"
      WHEN REGEXP_CONTAINS(LOWER(investment_type), r"(share & trading|trading|share|trade)") THEN "trading"
      ELSE "others"
    END AS grouped_investment_type
  FROM exploded_investments
),

-- Aggregation by user_id and year_month_date and investment type
aggregated_investments AS (
  SELECT 
    user_id,
    year_month_date,
    grouped_investment_type,
    SUM(investment_amount) AS total_investment_amount
  FROM grouped_investments
  GROUP BY user_id, year_month_date, grouped_investment_type
),

-- Aggregate totals over each time window per user and investment type
investment_windows AS (
  SELECT
    user_id,
    grouped_investment_type,
    SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows["last_1_months"]['start_date']}') AND DATE('{as_of_end_dt}') THEN total_investment_amount ELSE 0 END) AS last_1_month,
    SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows["last_3_months"]['start_date']}') AND DATE('{as_of_end_dt}') THEN total_investment_amount ELSE 0 END) AS last_3_months,
    SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows["last_6_months"]['start_date']}') AND DATE('{as_of_end_dt}') THEN total_investment_amount ELSE 0 END) AS last_6_months,
    SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows["last_12_months"]['start_date']}') AND DATE('{as_of_end_dt}') THEN total_investment_amount ELSE 0 END) AS last_12_months
    FROM aggregated_investments
    GROUP BY user_id, grouped_investment_type
),

-- Pivot the investment types to columns
pivoted_investments AS (
  SELECT
    user_id,
    MAX(IF(grouped_investment_type = 'fixed_deposit', last_1_month, 0)) AS fixed_deposit_1m,
    MAX(IF(grouped_investment_type = 'fixed_deposit', last_3_months, 0)) AS fixed_deposit_3m,
    MAX(IF(grouped_investment_type = 'fixed_deposit', last_6_months, 0)) AS fixed_deposit_6m,
    MAX(IF(grouped_investment_type = 'fixed_deposit', last_12_months, 0)) AS fixed_deposit_12m,

    MAX(IF(grouped_investment_type = 'recurring_deposit', last_1_month, 0)) AS recurring_deposit_1m,
    MAX(IF(grouped_investment_type = 'recurring_deposit', last_3_months, 0)) AS recurring_deposit_3m,
    MAX(IF(grouped_investment_type = 'recurring_deposit', last_6_months, 0)) AS recurring_deposit_6m,
    MAX(IF(grouped_investment_type = 'recurring_deposit', last_12_months, 0)) AS recurring_deposit_12m,

    MAX(IF(grouped_investment_type = 'term_deposit', last_1_month, 0)) AS term_deposit_1m,
    MAX(IF(grouped_investment_type = 'term_deposit', last_3_months, 0)) AS term_deposit_3m,
    MAX(IF(grouped_investment_type = 'term_deposit', last_6_months, 0)) AS term_deposit_6m,
    MAX(IF(grouped_investment_type = 'term_deposit', last_12_months, 0)) AS term_deposit_12m,

    MAX(IF(grouped_investment_type = 'provident_fund', last_1_month, 0)) AS provident_fund_1m,
    MAX(IF(grouped_investment_type = 'provident_fund', last_3_months, 0)) AS provident_fund_3m,
    MAX(IF(grouped_investment_type = 'provident_fund', last_6_months, 0)) AS provident_fund_6m,
    MAX(IF(grouped_investment_type = 'provident_fund', last_12_months, 0)) AS provident_fund_12m,

    MAX(IF(grouped_investment_type = 'nps', last_1_month, 0)) AS nps_1m,
    MAX(IF(grouped_investment_type = 'nps', last_3_months, 0)) AS nps_3m,
    MAX(IF(grouped_investment_type = 'nps', last_6_months, 0)) AS nps_6m,
    MAX(IF(grouped_investment_type = 'nps', last_12_months, 0)) AS nps_12m,

    MAX(IF(grouped_investment_type = 'mutual_fund', last_1_month, 0)) AS mutual_fund_1m,
    MAX(IF(grouped_investment_type = 'mutual_fund', last_3_months, 0)) AS mutual_fund_3m,
    MAX(IF(grouped_investment_type = 'mutual_fund', last_6_months, 0)) AS mutual_fund_6m,
    MAX(IF(grouped_investment_type = 'mutual_fund', last_12_months, 0)) AS mutual_fund_12m,

    MAX(IF(grouped_investment_type = 'trading', last_1_month, 0)) AS trading_1m,
    MAX(IF(grouped_investment_type = 'trading', last_3_months, 0)) AS trading_3m,
    MAX(IF(grouped_investment_type = 'trading', last_6_months, 0)) AS trading_6m,
    MAX(IF(grouped_investment_type = 'trading', last_12_months, 0)) AS trading_12m,

    MAX(IF(grouped_investment_type = 'others', last_1_month, 0)) AS others_1m,
    MAX(IF(grouped_investment_type = 'others', last_3_months, 0)) AS others_3m,
    MAX(IF(grouped_investment_type = 'others', last_6_months, 0)) AS others_6m,
    MAX(IF(grouped_investment_type = 'others', last_12_months, 0)) AS others_12m

  FROM investment_windows
  GROUP BY user_id
),

-- Calculate ratios 1/3, 3/6, 6/12, 3/12 per investment type
investment_ratios AS (
  SELECT
    user_id,

    -- Fixed Deposit Ratios
    SAFE_DIVIDE(fixed_deposit_1m, NULLIF(fixed_deposit_3m, 0)) AS fixed_deposit_1m_3m_ratio,
    SAFE_DIVIDE(fixed_deposit_3m, NULLIF(fixed_deposit_6m, 0)) AS fixed_deposit_3m_6m_ratio,
    SAFE_DIVIDE(fixed_deposit_6m, NULLIF(fixed_deposit_12m, 0)) AS fixed_deposit_6m_12m_ratio,
    SAFE_DIVIDE(fixed_deposit_3m, NULLIF(fixed_deposit_12m, 0)) AS fixed_deposit_3m_12m_ratio,

    -- Recurring Deposit Ratios
    SAFE_DIVIDE(recurring_deposit_1m, NULLIF(recurring_deposit_3m, 0)) AS recurring_deposit_1m_3m_ratio,
    SAFE_DIVIDE(recurring_deposit_3m, NULLIF(recurring_deposit_6m, 0)) AS recurring_deposit_3m_6m_ratio,
    SAFE_DIVIDE(recurring_deposit_6m, NULLIF(recurring_deposit_12m, 0)) AS recurring_deposit_6m_12m_ratio,
    SAFE_DIVIDE(recurring_deposit_3m, NULLIF(recurring_deposit_12m, 0)) AS recurring_deposit_3m_12m_ratio,

    -- Term Deposit Ratios
    SAFE_DIVIDE(term_deposit_1m, NULLIF(term_deposit_3m, 0)) AS term_deposit_1m_3m_ratio,
    SAFE_DIVIDE(term_deposit_3m, NULLIF(term_deposit_6m, 0)) AS term_deposit_3m_6m_ratio,
    SAFE_DIVIDE(term_deposit_6m, NULLIF(term_deposit_12m, 0)) AS term_deposit_6m_12m_ratio,
    SAFE_DIVIDE(term_deposit_3m, NULLIF(term_deposit_12m, 0)) AS term_deposit_3m_12m_ratio,

    -- Provident Fund Ratios
    SAFE_DIVIDE(provident_fund_1m, NULLIF(provident_fund_3m, 0)) AS provident_fund_1m_3m_ratio,
    SAFE_DIVIDE(provident_fund_3m, NULLIF(provident_fund_6m, 0)) AS provident_fund_3m_6m_ratio,
    SAFE_DIVIDE(provident_fund_6m, NULLIF(provident_fund_12m, 0)) AS provident_fund_6m_12m_ratio,
    SAFE_DIVIDE(provident_fund_3m, NULLIF(provident_fund_12m, 0)) AS provident_fund_3m_12m_ratio,

    -- NPS Ratios
    SAFE_DIVIDE(nps_1m, NULLIF(nps_3m, 0)) AS nps_1m_3m_ratio,
    SAFE_DIVIDE(nps_3m, NULLIF(nps_6m, 0)) AS nps_3m_6m_ratio,
    SAFE_DIVIDE(nps_6m, NULLIF(nps_12m, 0)) AS nps_6m_12m_ratio,
    SAFE_DIVIDE(nps_3m, NULLIF(nps_12m, 0)) AS nps_3m_12m_ratio,

    -- Mutual Fund Ratios
    SAFE_DIVIDE(mutual_fund_1m, NULLIF(mutual_fund_3m, 0)) AS mutual_fund_1m_3m_ratio,
    SAFE_DIVIDE(mutual_fund_3m, NULLIF(mutual_fund_6m, 0)) AS mutual_fund_3m_6m_ratio,
    SAFE_DIVIDE(mutual_fund_6m, NULLIF(mutual_fund_12m, 0)) AS mutual_fund_6m_12m_ratio,
    SAFE_DIVIDE(mutual_fund_3m, NULLIF(mutual_fund_12m, 0)) AS mutual_fund_3m_12m_ratio,

    -- Trading Ratios
    SAFE_DIVIDE(trading_1m, NULLIF(trading_3m, 0)) AS trading_1m_3m_ratio,
    SAFE_DIVIDE(trading_3m, NULLIF(trading_6m, 0)) AS trading_3m_6m_ratio,
    SAFE_DIVIDE(trading_6m, NULLIF(trading_12m, 0)) AS trading_6m_12m_ratio,
    SAFE_DIVIDE(trading_3m, NULLIF(trading_12m, 0)) AS trading_3m_12m_ratio,

    -- Others Ratios
    SAFE_DIVIDE(others_1m, NULLIF(others_3m, 0)) AS others_1m_3m_ratio,
    SAFE_DIVIDE(others_3m, NULLIF(others_6m, 0)) AS others_3m_6m_ratio,
    SAFE_DIVIDE(others_6m, NULLIF(others_12m, 0)) AS others_6m_12m_ratio,
    SAFE_DIVIDE(others_3m, NULLIF(others_12m, 0)) AS others_3m_12m_ratio

  FROM pivoted_investments
),

-- Min-Max Scaling over last 12 months 
min_max_scaled AS (
  SELECT
    user_id,

    -- 1 Month
    SAFE_DIVIDE(fixed_deposit_1m - MIN(fixed_deposit_1m) OVER (), NULLIF(MAX(fixed_deposit_1m) OVER () - MIN(fixed_deposit_1m) OVER (), 0)) AS fixed_deposit_1m_scaled,
    SAFE_DIVIDE(recurring_deposit_1m - MIN(recurring_deposit_1m) OVER (), NULLIF(MAX(recurring_deposit_1m) OVER () - MIN(recurring_deposit_1m) OVER (), 0)) AS recurring_deposit_1m_scaled,
    SAFE_DIVIDE(term_deposit_1m - MIN(term_deposit_1m) OVER (), NULLIF(MAX(term_deposit_1m) OVER () - MIN(term_deposit_1m) OVER (), 0)) AS term_deposit_1m_scaled,
    SAFE_DIVIDE(provident_fund_1m - MIN(provident_fund_1m) OVER (), NULLIF(MAX(provident_fund_1m) OVER () - MIN(provident_fund_1m) OVER (), 0)) AS provident_fund_1m_scaled,
    SAFE_DIVIDE(nps_1m - MIN(nps_1m) OVER (), NULLIF(MAX(nps_1m) OVER () - MIN(nps_1m) OVER (), 0)) AS nps_1m_scaled,
    SAFE_DIVIDE(mutual_fund_1m - MIN(mutual_fund_1m) OVER (), NULLIF(MAX(mutual_fund_1m) OVER () - MIN(mutual_fund_1m) OVER (), 0)) AS mutual_fund_1m_scaled,
    SAFE_DIVIDE(trading_1m - MIN(trading_1m) OVER (), NULLIF(MAX(trading_1m) OVER () - MIN(trading_1m) OVER (), 0)) AS trading_1m_scaled,
    SAFE_DIVIDE(others_1m - MIN(others_1m) OVER (), NULLIF(MAX(others_1m) OVER () - MIN(others_1m) OVER (), 0)) AS others_1m_scaled,

    -- 3 Month
    SAFE_DIVIDE(fixed_deposit_3m - MIN(fixed_deposit_3m) OVER (), NULLIF(MAX(fixed_deposit_3m) OVER () - MIN(fixed_deposit_3m) OVER (), 0)) AS fixed_deposit_3m_scaled,
    SAFE_DIVIDE(recurring_deposit_3m - MIN(recurring_deposit_3m) OVER (), NULLIF(MAX(recurring_deposit_3m) OVER () - MIN(recurring_deposit_3m) OVER (), 0)) AS recurring_deposit_3m_scaled,
    SAFE_DIVIDE(term_deposit_3m - MIN(term_deposit_3m) OVER (), NULLIF(MAX(term_deposit_3m) OVER () - MIN(term_deposit_3m) OVER (), 0)) AS term_deposit_3m_scaled,
    SAFE_DIVIDE(provident_fund_3m - MIN(provident_fund_3m) OVER (), NULLIF(MAX(provident_fund_3m) OVER () - MIN(provident_fund_3m) OVER (), 0)) AS provident_fund_3m_scaled,
    SAFE_DIVIDE(nps_3m - MIN(nps_3m) OVER (), NULLIF(MAX(nps_3m) OVER () - MIN(nps_3m) OVER (), 0)) AS nps_3m_scaled,
    SAFE_DIVIDE(mutual_fund_3m - MIN(mutual_fund_3m) OVER (), NULLIF(MAX(mutual_fund_3m) OVER () - MIN(mutual_fund_3m) OVER (), 0)) AS mutual_fund_3m_scaled,
    SAFE_DIVIDE(trading_3m - MIN(trading_3m) OVER (), NULLIF(MAX(trading_3m) OVER () - MIN(trading_3m) OVER (), 0)) AS trading_3m_scaled,
    SAFE_DIVIDE(others_3m - MIN(others_3m) OVER (), NULLIF(MAX(others_3m) OVER () - MIN(others_3m) OVER (), 0)) AS others_3m_scaled,

    -- 6 Month
    SAFE_DIVIDE(fixed_deposit_6m - MIN(fixed_deposit_6m) OVER (), NULLIF(MAX(fixed_deposit_6m) OVER () - MIN(fixed_deposit_6m) OVER (), 0)) AS fixed_deposit_6m_scaled,
    SAFE_DIVIDE(recurring_deposit_6m - MIN(recurring_deposit_6m) OVER (), NULLIF(MAX(recurring_deposit_6m) OVER () - MIN(recurring_deposit_6m) OVER (), 0)) AS recurring_deposit_6m_scaled,
    SAFE_DIVIDE(term_deposit_6m - MIN(term_deposit_6m) OVER (), NULLIF(MAX(term_deposit_6m) OVER () - MIN(term_deposit_6m) OVER (), 0)) AS term_deposit_6m_scaled,
    SAFE_DIVIDE(provident_fund_6m - MIN(provident_fund_6m) OVER (), NULLIF(MAX(provident_fund_6m) OVER () - MIN(provident_fund_6m) OVER (), 0)) AS provident_fund_6m_scaled,
    SAFE_DIVIDE(nps_6m - MIN(nps_6m) OVER (), NULLIF(MAX(nps_6m) OVER () - MIN(nps_6m) OVER (), 0)) AS nps_6m_scaled,
    SAFE_DIVIDE(mutual_fund_6m - MIN(mutual_fund_6m) OVER (), NULLIF(MAX(mutual_fund_6m) OVER () - MIN(mutual_fund_6m) OVER (), 0)) AS mutual_fund_6m_scaled,
    SAFE_DIVIDE(trading_6m - MIN(trading_6m) OVER (), NULLIF(MAX(trading_6m) OVER () - MIN(trading_6m) OVER (), 0)) AS trading_6m_scaled,
    SAFE_DIVIDE(others_6m - MIN(others_6m) OVER (), NULLIF(MAX(others_6m) OVER () - MIN(others_6m) OVER (), 0)) AS others_6m_scaled,
    
    
    -- 12 Month
    SAFE_DIVIDE(fixed_deposit_12m - MIN(fixed_deposit_12m) OVER (), NULLIF(MAX(fixed_deposit_12m) OVER () - MIN(fixed_deposit_12m) OVER (), 0)) AS fixed_deposit_12m_scaled,
    SAFE_DIVIDE(recurring_deposit_12m - MIN(recurring_deposit_12m) OVER (), NULLIF(MAX(recurring_deposit_12m) OVER () - MIN(recurring_deposit_12m) OVER (), 0)) AS recurring_deposit_12m_scaled,
    SAFE_DIVIDE(term_deposit_12m - MIN(term_deposit_12m) OVER (), NULLIF(MAX(term_deposit_12m) OVER () - MIN(term_deposit_12m) OVER (), 0)) AS term_deposit_12m_scaled,
    SAFE_DIVIDE(provident_fund_12m - MIN(provident_fund_12m) OVER (), NULLIF(MAX(provident_fund_12m) OVER () - MIN(provident_fund_12m) OVER (), 0)) AS provident_fund_12m_scaled,
    SAFE_DIVIDE(nps_12m - MIN(nps_12m) OVER (), NULLIF(MAX(nps_12m) OVER () - MIN(nps_12m) OVER (), 0)) AS nps_12m_scaled,
    SAFE_DIVIDE(mutual_fund_12m - MIN(mutual_fund_12m) OVER (), NULLIF(MAX(mutual_fund_12m) OVER () - MIN(mutual_fund_12m) OVER (), 0)) AS mutual_fund_12m_scaled,
    SAFE_DIVIDE(trading_12m - MIN(trading_12m) OVER (), NULLIF(MAX(trading_12m) OVER () - MIN(trading_12m) OVER (), 0)) AS trading_12m_scaled,
    SAFE_DIVIDE(others_12m - MIN(others_12m) OVER (), NULLIF(MAX(others_12m) OVER () - MIN(others_12m) OVER (), 0)) AS others_12m_scaled

  FROM pivoted_investments
)

SELECT 
  p.user_id,

  -- Raw Aggregations
  fixed_deposit_1m, fixed_deposit_3m, fixed_deposit_6m, fixed_deposit_12m,
  recurring_deposit_1m, recurring_deposit_3m, recurring_deposit_6m, recurring_deposit_12m,
  term_deposit_1m, term_deposit_3m, term_deposit_6m, term_deposit_12m,
  provident_fund_1m, provident_fund_3m, provident_fund_6m, provident_fund_12m,
  nps_1m, nps_3m, nps_6m, nps_12m,
  mutual_fund_1m, mutual_fund_3m, mutual_fund_6m, mutual_fund_12m,
  trading_1m, trading_3m, trading_6m, trading_12m,
  others_1m, others_3m, others_6m, others_12m,

  -- Ratios
  r.fixed_deposit_1m_3m_ratio, r.fixed_deposit_3m_6m_ratio, r.fixed_deposit_6m_12m_ratio, r.fixed_deposit_3m_12m_ratio,
  r.recurring_deposit_1m_3m_ratio, r.recurring_deposit_3m_6m_ratio, r.recurring_deposit_6m_12m_ratio, r.recurring_deposit_3m_12m_ratio,
  r.term_deposit_1m_3m_ratio, r.term_deposit_3m_6m_ratio, r.term_deposit_6m_12m_ratio, r.term_deposit_3m_12m_ratio,
  r.provident_fund_1m_3m_ratio, r.provident_fund_3m_6m_ratio, r.provident_fund_6m_12m_ratio, r.provident_fund_3m_12m_ratio,
  r.nps_1m_3m_ratio, r.nps_3m_6m_ratio, r.nps_6m_12m_ratio, r.nps_3m_12m_ratio,
  r.mutual_fund_1m_3m_ratio, r.mutual_fund_3m_6m_ratio, r.mutual_fund_6m_12m_ratio, r.mutual_fund_3m_12m_ratio,
  r.trading_1m_3m_ratio, r.trading_3m_6m_ratio, r.trading_6m_12m_ratio, r.trading_3m_12m_ratio,
  r.others_1m_3m_ratio, r.others_3m_6m_ratio, r.others_6m_12m_ratio, r.others_3m_12m_ratio,

  -- Scaled 1 Month
  m.fixed_deposit_1m_scaled,
  m.recurring_deposit_1m_scaled,
  m.term_deposit_1m_scaled,
  m.provident_fund_1m_scaled,
  m.nps_1m_scaled,
  m.mutual_fund_1m_scaled,
  m.trading_1m_scaled,
  m.others_1m_scaled,

 -- Scaled 3 Month
  m.fixed_deposit_3m_scaled,
  m.recurring_deposit_3m_scaled,
  m.term_deposit_3m_scaled,
  m.provident_fund_3m_scaled,
  m.nps_3m_scaled,
  m.mutual_fund_3m_scaled,
  m.trading_3m_scaled,
  m.others_3m_scaled,

 -- Scaled 6 Month
  m.fixed_deposit_6m_scaled,
  m.recurring_deposit_6m_scaled,
  m.term_deposit_6m_scaled,
  m.provident_fund_6m_scaled,
  m.nps_6m_scaled,
  m.mutual_fund_6m_scaled,
  m.trading_6m_scaled,
  m.others_6m_scaled,

 -- Scaled 12 Month
  m.fixed_deposit_12m_scaled,
  m.recurring_deposit_12m_scaled,
  m.term_deposit_12m_scaled,
  m.provident_fund_12m_scaled,
  m.nps_12m_scaled,
  m.mutual_fund_12m_scaled,
  m.trading_12m_scaled,
  m.others_12m_scaled

FROM pivoted_investments p
LEFT JOIN investment_ratios r USING(user_id)
LEFT JOIN min_max_scaled m USING(user_id)
ORDER BY user_id
;
"""

# Execute the query and load the result into a DataFrame
inv_df = client.query(inv_query).to_dataframe()

In [24]:
inv_df = inv_df.drop(columns=[col for col in inv_df.columns if col.lower().startswith('others')])
inv_df.rename(columns={'user_id': 'mobilenumber'}, inplace=True)
inv_df = inv_df.round(2)
inv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 922809 entries, 0 to 922808
Data columns (total 85 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   mobilenumber                    922808 non-null  Int64  
 1   fixed_deposit_1m                922809 non-null  float64
 2   fixed_deposit_3m                922809 non-null  float64
 3   fixed_deposit_6m                922809 non-null  float64
 4   fixed_deposit_12m               922809 non-null  float64
 5   recurring_deposit_1m            922809 non-null  float64
 6   recurring_deposit_3m            922809 non-null  float64
 7   recurring_deposit_6m            922809 non-null  float64
 8   recurring_deposit_12m           922809 non-null  float64
 9   term_deposit_1m                 922809 non-null  float64
 10  term_deposit_3m                 922809 non-null  float64
 11  term_deposit_6m                 922809 non-null  float64
 12  term_deposit_12m

In [25]:
inv_df.mobilenumber.nunique(),inv_df.shape[0]

(922808, 922809)

In [26]:
# merging
model_base = model_base.merge(inv_df, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5077450 entries, 0 to 5077449
Data columns (total 86 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   mobilenumber                    int64  
 1   customer_id                     Int64  
 2   fixed_deposit_1m                float64
 3   fixed_deposit_3m                float64
 4   fixed_deposit_6m                float64
 5   fixed_deposit_12m               float64
 6   recurring_deposit_1m            float64
 7   recurring_deposit_3m            float64
 8   recurring_deposit_6m            float64
 9   recurring_deposit_12m           float64
 10  term_deposit_1m                 float64
 11  term_deposit_3m                 float64
 12  term_deposit_6m                 float64
 13  term_deposit_12m                float64
 14  provident_fund_1m               float64
 15  provident_fund_3m               float64
 16  provident_fund_6m               float64
 17  provident_fund_12m         

In [27]:
model_base.head()

,mobilenumber,customer_id,fixed_deposit_1m,fixed_deposit_3m,fixed_deposit_6m,fixed_deposit_12m,recurring_deposit_1m,recurring_deposit_3m,recurring_deposit_6m,recurring_deposit_12m,term_deposit_1m,term_deposit_3m,term_deposit_6m,term_deposit_12m,provident_fund_1m,provident_fund_3m,provident_fund_6m,provident_fund_12m,nps_1m,nps_3m,nps_6m,nps_12m,mutual_fund_1m,mutual_fund_3m,mutual_fund_6m,mutual_fund_12m,trading_1m,trading_3m,trading_6m,trading_12m,fixed_deposit_1m_3m_ratio,fixed_deposit_3m_6m_ratio,fixed_deposit_6m_12m_ratio,fixed_deposit_3m_12m_ratio,recurring_deposit_1m_3m_ratio,recurring_deposit_3m_6m_ratio,recurring_deposit_6m_12m_ratio,recurring_deposit_3m_12m_ratio,term_deposit_1m_3m_ratio,term_deposit_3m_6m_ratio,term_deposit_6m_12m_ratio,term_deposit_3m_12m_ratio,provident_fund_1m_3m_ratio,provident_fund_3m_6m_ratio,provident_fund_6m_12m_ratio,provident_fund_3m_12m_ratio,nps_1m_3m_ratio,nps_3m_6m_ratio,nps_6m_12m_ratio,nps_3m_12m_ratio,mutual_fund_1m_3m_ratio,mutual_fund_3m_6m_ratio,mutual_fund_6m_12m_ratio,mutual_fund_3m_12m_ratio,trading_1m_3m_ratio,trading_3m_6m_ratio,trading_6m_12m_ratio,trading_3m_12m_ratio,fixed_deposit_1m_scaled,recurring_deposit_1m_scaled,term_deposit_1m_scaled,provident_fund_1m_scaled,nps_1m_scaled,mutual_fund_1m_scaled,trading_1m_scaled,fixed_deposit_3m_scaled,recurring_deposit_3m_scaled,term_deposit_3m_scaled,provident_fund_3m_scaled,nps_3m_scaled,mutual_fund_3m_scaled,trading_3m_scaled,fixed_deposit_6m_scaled,recurring_deposit_6m_scaled,term_deposit_6m_scaled,provident_fund_6m_scaled,nps_6m_scaled,mutual_fund_6m_scaled,trading_6m_scaled,fixed_deposit_12m_scaled,recurring_deposit_12m_scaled,term_deposit_12m_scaled,provident_fund_12m_scaled,nps_12m_scaled,mutual_fund_12m_scaled,trading_12m_scaled
0,9359701533,2953643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8940337760,3961083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7510026885,3962064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9704105644,3964445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9625179172,3958561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
as_of_end_dt

Timestamp('2025-03-31 00:00:00')

# salary feature

In [29]:
salary_query = f"""
-- Use as_of_end_dt from Python
WITH max_date_cte AS (
  SELECT DATE('{as_of_end_dt}') AS max_date
),

-- Filter salary entries per time windows
filtered_salary AS (
  SELECT
    SAFE_CAST(RIGHT(REPLACE(user_id, ',', ''), 10) AS INT64) AS user_id,
    SAFE_CAST(REPLACE(total_salary_credit, ',', '') AS FLOAT64) AS salary,
    year_month_date
  FROM `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed`,
       max_date_cte
  WHERE year_month_date BETWEEN DATE('{date_windows['last_12_months']['start_date']}') AND (SELECT max_date FROM max_date_cte)
),

-- Aggregate salary for all 4 time windows
aggregated_salary AS (
  SELECT
    user_id,
    MAX(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_1_months']['start_date']}') AND DATE('{date_windows['last_1_months']['end_date']}') THEN salary ELSE NULL END) AS salary_1m,
    MAX(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_3_months']['start_date']}') AND DATE('{date_windows['last_3_months']['end_date']}') THEN salary ELSE NULL END) AS salary_3m,
    MAX(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_6_months']['start_date']}') AND DATE('{date_windows['last_6_months']['end_date']}') THEN salary ELSE NULL END) AS salary_6m,
    MAX(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_12_months']['start_date']}') AND DATE('{date_windows['last_12_months']['end_date']}') THEN salary ELSE NULL END) AS salary_12m
  FROM filtered_salary
  GROUP BY user_id
)

-- Final clean select
SELECT
  user_id,
  CASE WHEN salary_1m = 0.0 THEN NULL ELSE salary_1m END AS salary_1m,
  CASE WHEN salary_3m = 0.0 THEN NULL ELSE salary_3m END AS salary_3m,
  CASE WHEN salary_6m = 0.0 THEN NULL ELSE salary_6m END AS salary_6m,
  CASE WHEN salary_12m = 0.0 THEN NULL ELSE salary_12m END AS salary_12m
FROM aggregated_salary
"""
# Run the query and get a Pandas DataFrame
aggregated_salary_df = client.query(salary_query).to_dataframe()
aggregated_salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884462 entries, 0 to 4884461
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     Int64  
 1   salary_1m   float64
 2   salary_3m   float64
 3   salary_6m   float64
 4   salary_12m  float64
dtypes: Int64(1), float64(4)
memory usage: 191.0 MB


In [30]:
aggregated_salary_df.rename(columns={'user_id': 'mobilenumber'}, inplace=True)

In [31]:
aggregated_salary_df.sample(5)

,mobilenumber,salary_1m,salary_3m,salary_6m,salary_12m
3366427,7014709267,NaN,NaN,NaN,NaN
3775842,9726853576,NaN,NaN,NaN,NaN
4095958,8624040156,NaN,NaN,NaN,NaN
1336967,7909162631,NaN,NaN,NaN,NaN
398839,7665340408,NaN,NaN,NaN,NaN


In [32]:
# merging
model_base = model_base.merge(aggregated_salary_df, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5077450 entries, 0 to 5077449
Data columns (total 90 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   mobilenumber                    int64  
 1   customer_id                     Int64  
 2   fixed_deposit_1m                float64
 3   fixed_deposit_3m                float64
 4   fixed_deposit_6m                float64
 5   fixed_deposit_12m               float64
 6   recurring_deposit_1m            float64
 7   recurring_deposit_3m            float64
 8   recurring_deposit_6m            float64
 9   recurring_deposit_12m           float64
 10  term_deposit_1m                 float64
 11  term_deposit_3m                 float64
 12  term_deposit_6m                 float64
 13  term_deposit_12m                float64
 14  provident_fund_1m               float64
 15  provident_fund_3m               float64
 16  provident_fund_6m               float64
 17  provident_fund_12m         

# spending behaviour

In [33]:
QUER1 = f"""


WITH max_date_cte AS (
  SELECT DATE('{as_of_end_dt}') AS max_date
),

FlatSpends AS (
    SELECT 
        SAFE_CAST(RIGHT(REPLACE(user_id, ',', ''), 10) AS INT64) AS user_id,
        year_month_date,
        LOWER(b.category) AS category,
        ABS(SAFE_CAST(REPLACE(b.amount, ',', '') AS FLOAT64)) AS spend_amount
    FROM 
        `abcd-dataplatform-prod.abcd_digitap_transformed.users_digitap_sms_data_transformed` a,
        UNNEST(top_spend_areas) AS b
    WHERE 
        year_month_date BETWEEN DATE('{date_windows['last_12_months']['start_date']}') AND (SELECT max_date FROM max_date_cte)
),

Aggregated AS (
    SELECT
        user_id,
        category,

        -- Total spend across all time (last 12 months window)
        SUM(spend_amount) AS amt_total_spend,

        -- 1 Month Window
        SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_1_months']['start_date']}') AND DATE('{date_windows['last_1_months']['end_date']}') THEN spend_amount ELSE 0 END) AS amt_last_1m,

        -- 3 Month Window
        SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_3_months']['start_date']}') AND DATE('{date_windows['last_3_months']['end_date']}') THEN spend_amount ELSE 0 END) AS amt_last_3m,

        -- 6 Month Window
        SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_6_months']['start_date']}') AND DATE('{date_windows['last_6_months']['end_date']}') THEN spend_amount ELSE 0 END) AS amt_last_6m,

        -- 12 Month Window
        SUM(CASE WHEN year_month_date BETWEEN DATE('{date_windows['last_12_months']['start_date']}') AND DATE('{date_windows['last_12_months']['end_date']}') THEN spend_amount ELSE 0 END) AS amt_last_12m

    FROM FlatSpends
    GROUP BY user_id, category
),

Pivoted AS (
    SELECT
        user_id,

        -- Total spend all time
        SUM(CASE WHEN category = 'insurance' THEN amt_total_spend ELSE 0 END) AS amt_insurance_spend,
        SUM(CASE WHEN category = 'investment' THEN amt_total_spend ELSE 0 END) AS amt_investment_spend,
        SUM(CASE WHEN category = 'medical' THEN amt_total_spend ELSE 0 END) AS amt_medical_spend,
        SUM(CASE WHEN category = 'bills & utilities' THEN amt_total_spend ELSE 0 END) AS amt_bills_utilities_spend,
        SUM(CASE WHEN category = 'shopping & purchase' THEN amt_total_spend ELSE 0 END) AS amt_shopping_purchase_spend,
        SUM(CASE WHEN category = 'online shopping' THEN amt_total_spend ELSE 0 END) AS amt_online_shopping_spend,
        SUM(CASE WHEN category = 'entertainment' THEN amt_total_spend ELSE 0 END) AS amt_entertainment_spend,
        SUM(CASE WHEN category = 'groceries' THEN amt_total_spend ELSE 0 END) AS amt_groceries_spend,
        SUM(CASE WHEN category = 'education' THEN amt_total_spend ELSE 0 END) AS amt_education_spend,
        SUM(CASE WHEN category = 'travel' THEN amt_total_spend ELSE 0 END) AS amt_travel_spend,
        SUM(CASE WHEN category = 'emi repayment' THEN amt_total_spend ELSE 0 END) AS amt_emi_repayment_spend,
        SUM(CASE WHEN category = 'housing services' THEN amt_total_spend ELSE 0 END) AS amt_housing_services_spend,
        SUM(CASE WHEN category = 'vehicle expenses' THEN amt_total_spend ELSE 0 END) AS amt_vehicle_expenses_spend,

        -- 1 Month Window
        SUM(CASE WHEN category = 'insurance' THEN amt_last_1m ELSE 0 END) AS amt_insurance_spend_1m,
        SUM(CASE WHEN category = 'investment' THEN amt_last_1m ELSE 0 END) AS amt_investment_spend_1m,
        SUM(CASE WHEN category = 'medical' THEN amt_last_1m ELSE 0 END) AS amt_medical_spend_1m,
        SUM(CASE WHEN category = 'bills & utilities' THEN amt_last_1m ELSE 0 END) AS amt_bills_utilities_spend_1m,
        SUM(CASE WHEN category = 'shopping & purchase' THEN amt_last_1m ELSE 0 END) AS amt_shopping_purchase_spend_1m,
        SUM(CASE WHEN category = 'online shopping' THEN amt_last_1m ELSE 0 END) AS amt_online_shopping_spend_1m,
        SUM(CASE WHEN category = 'entertainment' THEN amt_last_1m ELSE 0 END) AS amt_entertainment_spend_1m,
        SUM(CASE WHEN category = 'groceries' THEN amt_last_1m ELSE 0 END) AS amt_groceries_spend_1m,
        SUM(CASE WHEN category = 'education' THEN amt_last_1m ELSE 0 END) AS amt_education_spend_1m,
        SUM(CASE WHEN category = 'travel' THEN amt_last_1m ELSE 0 END) AS amt_travel_spend_1m,
        SUM(CASE WHEN category = 'emi repayment' THEN amt_last_1m ELSE 0 END) AS amt_emi_repayment_spend_1m,
        SUM(CASE WHEN category = 'housing services' THEN amt_last_1m ELSE 0 END) AS amt_housing_services_spend_1m,
        SUM(CASE WHEN category = 'vehicle expenses' THEN amt_last_1m ELSE 0 END) AS amt_vehicle_expenses_spend_1m,

        -- 3 Month Window
        SUM(CASE WHEN category = 'insurance' THEN amt_last_3m ELSE 0 END) AS amt_insurance_spend_3m,
        SUM(CASE WHEN category = 'investment' THEN amt_last_3m ELSE 0 END) AS amt_investment_spend_3m,
        SUM(CASE WHEN category = 'medical' THEN amt_last_3m ELSE 0 END) AS amt_medical_spend_3m,
        SUM(CASE WHEN category = 'bills & utilities' THEN amt_last_3m ELSE 0 END) AS amt_bills_utilities_spend_3m,
        SUM(CASE WHEN category = 'shopping & purchase' THEN amt_last_3m ELSE 0 END) AS amt_shopping_purchase_spend_3m,
        SUM(CASE WHEN category = 'online shopping' THEN amt_last_3m ELSE 0 END) AS amt_online_shopping_spend_3m,
        SUM(CASE WHEN category = 'entertainment' THEN amt_last_3m ELSE 0 END) AS amt_entertainment_spend_3m,
        SUM(CASE WHEN category = 'groceries' THEN amt_last_3m ELSE 0 END) AS amt_groceries_spend_3m,
        SUM(CASE WHEN category = 'education' THEN amt_last_3m ELSE 0 END) AS amt_education_spend_3m,
        SUM(CASE WHEN category = 'travel' THEN amt_last_3m ELSE 0 END) AS amt_travel_spend_3m,
        SUM(CASE WHEN category = 'emi repayment' THEN amt_last_3m ELSE 0 END) AS amt_emi_repayment_spend_3m,
        SUM(CASE WHEN category = 'housing services' THEN amt_last_3m ELSE 0 END) AS amt_housing_services_spend_3m,
        SUM(CASE WHEN category = 'vehicle expenses' THEN amt_last_3m ELSE 0 END) AS amt_vehicle_expenses_spend_3m,

        -- 6 Month Window
        SUM(CASE WHEN category = 'insurance' THEN amt_last_6m ELSE 0 END) AS amt_insurance_spend_6m,
        SUM(CASE WHEN category = 'investment' THEN amt_last_6m ELSE 0 END) AS amt_investment_spend_6m,
        SUM(CASE WHEN category = 'medical' THEN amt_last_6m ELSE 0 END) AS amt_medical_spend_6m,
        SUM(CASE WHEN category = 'bills & utilities' THEN amt_last_6m ELSE 0 END) AS amt_bills_utilities_spend_6m,
        SUM(CASE WHEN category = 'shopping & purchase' THEN amt_last_6m ELSE 0 END) AS amt_shopping_purchase_spend_6m,
        SUM(CASE WHEN category = 'online shopping' THEN amt_last_6m ELSE 0 END) AS amt_online_shopping_spend_6m,
        SUM(CASE WHEN category = 'entertainment' THEN amt_last_6m ELSE 0 END) AS amt_entertainment_spend_6m,
        SUM(CASE WHEN category = 'groceries' THEN amt_last_6m ELSE 0 END) AS amt_groceries_spend_6m,
        SUM(CASE WHEN category = 'education' THEN amt_last_6m ELSE 0 END) AS amt_education_spend_6m,
        SUM(CASE WHEN category = 'travel' THEN amt_last_6m ELSE 0 END) AS amt_travel_spend_6m,
        SUM(CASE WHEN category = 'emi repayment' THEN amt_last_6m ELSE 0 END) AS amt_emi_repayment_spend_6m,
        SUM(CASE WHEN category = 'housing services' THEN amt_last_6m ELSE 0 END) AS amt_housing_services_spend_6m,
        SUM(CASE WHEN category = 'vehicle expenses' THEN amt_last_6m ELSE 0 END) AS amt_vehicle_expenses_spend_6m,

        -- 12 Month Window
        SUM(CASE WHEN category = 'insurance' THEN amt_last_12m ELSE 0 END) AS amt_insurance_spend_12m,
        SUM(CASE WHEN category = 'investment' THEN amt_last_12m ELSE 0 END) AS amt_investment_spend_12m,
        SUM(CASE WHEN category = 'medical' THEN amt_last_12m ELSE 0 END) AS amt_medical_spend_12m,
        SUM(CASE WHEN category = 'bills & utilities' THEN amt_last_12m ELSE 0 END) AS amt_bills_utilities_spend_12m,
        SUM(CASE WHEN category = 'shopping & purchase' THEN amt_last_12m ELSE 0 END) AS amt_shopping_purchase_spend_12m,
        SUM(CASE WHEN category = 'online shopping' THEN amt_last_12m ELSE 0 END) AS amt_online_shopping_spend_12m,
        SUM(CASE WHEN category = 'entertainment' THEN amt_last_12m ELSE 0 END) AS amt_entertainment_spend_12m,
        SUM(CASE WHEN category = 'groceries' THEN amt_last_12m ELSE 0 END) AS amt_groceries_spend_12m,
        SUM(CASE WHEN category = 'education' THEN amt_last_12m ELSE 0 END) AS amt_education_spend_12m,
        SUM(CASE WHEN category = 'travel' THEN amt_last_12m ELSE 0 END) AS amt_travel_spend_12m,
        SUM(CASE WHEN category = 'emi repayment' THEN amt_last_12m ELSE 0 END) AS amt_emi_repayment_spend_12m,
        SUM(CASE WHEN category = 'housing services' THEN amt_last_12m ELSE 0 END) AS amt_housing_services_spend_12m,
        SUM(CASE WHEN category = 'vehicle expenses' THEN amt_last_12m ELSE 0 END) AS amt_vehicle_expenses_spend_12m

    FROM Aggregated
    GROUP BY user_id
)

SELECT * FROM Pivoted;
"""
spend_df = client.query(QUER1).to_dataframe()
spend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308493 entries, 0 to 4308492
Data columns (total 66 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   user_id                          Int64  
 1   amt_insurance_spend              float64
 2   amt_investment_spend             float64
 3   amt_medical_spend                float64
 4   amt_bills_utilities_spend        float64
 5   amt_shopping_purchase_spend      float64
 6   amt_online_shopping_spend        float64
 7   amt_entertainment_spend          float64
 8   amt_groceries_spend              float64
 9   amt_education_spend              float64
 10  amt_travel_spend                 float64
 11  amt_emi_repayment_spend          float64
 12  amt_housing_services_spend       float64
 13  amt_vehicle_expenses_spend       float64
 14  amt_insurance_spend_1m           float64
 15  amt_investment_spend_1m          float64
 16  amt_medical_spend_1m             float64
 17  amt_bill

In [34]:
spend_df.rename(columns={'user_id':'mobilenumber'},inplace=True)

In [35]:
# feature_list = [
#     "amt_insurance_spend",
#     "amt_investment_spend",
#     "amt_medical_spend",
#     "amt_bills_utilities_spend",
#     "amt_shopping_purchase_spend",
#     "amt_online_shopping_spend",
#     "amt_entertainment_spend",
#     "amt_groceries_spend",
#     "amt_education_spend",
#     "amt_travel_spend",
#     "amt_emi_repayment_spend",
#     "amt_housing_services_spend",
#     "amt_vehicle_expenses_spend"
# ]
# spend_df.drop(feature_list,axis=1,inplace=True)
# spend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308493 entries, 0 to 4308492
Data columns (total 53 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   mobilenumber                     Int64  
 1   amt_insurance_spend_1m           float64
 2   amt_investment_spend_1m          float64
 3   amt_medical_spend_1m             float64
 4   amt_bills_utilities_spend_1m     float64
 5   amt_shopping_purchase_spend_1m   float64
 6   amt_online_shopping_spend_1m     float64
 7   amt_entertainment_spend_1m       float64
 8   amt_groceries_spend_1m           float64
 9   amt_education_spend_1m           float64
 10  amt_travel_spend_1m              float64
 11  amt_emi_repayment_spend_1m       float64
 12  amt_housing_services_spend_1m    float64
 13  amt_vehicle_expenses_spend_1m    float64
 14  amt_insurance_spend_3m           float64
 15  amt_investment_spend_3m          float64
 16  amt_medical_spend_3m             float64
 17  amt_bill

In [36]:
# merging
model_base = model_base.merge(spend_df, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5077450 entries, 0 to 5077449
Columns: 142 entries, mobilenumber to amt_vehicle_expenses_spend_12m
dtypes: Int64(1), float64(140), int64(1)
memory usage: 5.4 GB


In [37]:
model_base.head()

,mobilenumber,customer_id,fixed_deposit_1m,fixed_deposit_3m,fixed_deposit_6m,fixed_deposit_12m,recurring_deposit_1m,recurring_deposit_3m,recurring_deposit_6m,recurring_deposit_12m,term_deposit_1m,term_deposit_3m,term_deposit_6m,term_deposit_12m,provident_fund_1m,provident_fund_3m,provident_fund_6m,provident_fund_12m,nps_1m,nps_3m,nps_6m,nps_12m,mutual_fund_1m,mutual_fund_3m,mutual_fund_6m,mutual_fund_12m,trading_1m,trading_3m,trading_6m,trading_12m,fixed_deposit_1m_3m_ratio,fixed_deposit_3m_6m_ratio,fixed_deposit_6m_12m_ratio,fixed_deposit_3m_12m_ratio,recurring_deposit_1m_3m_ratio,recurring_deposit_3m_6m_ratio,recurring_deposit_6m_12m_ratio,recurring_deposit_3m_12m_ratio,term_deposit_1m_3m_ratio,term_deposit_3m_6m_ratio,term_deposit_6m_12m_ratio,term_deposit_3m_12m_ratio,provident_fund_1m_3m_ratio,provident_fund_3m_6m_ratio,provident_fund_6m_12m_ratio,provident_fund_3m_12m_ratio,nps_1m_3m_ratio,nps_3m_6m_ratio,nps_6m_12m_ratio,nps_3m_12m_ratio,mutual_fund_1m_3m_ratio,mutual_fund_3m_6m_ratio,mutual_fund_6m_12m_ratio,mutual_fund_3m_12m_ratio,trading_1m_3m_ratio,trading_3m_6m_ratio,trading_6m_12m_ratio,trading_3m_12m_ratio,fixed_deposit_1m_scaled,recurring_deposit_1m_scaled,term_deposit_1m_scaled,provident_fund_1m_scaled,nps_1m_scaled,mutual_fund_1m_scaled,trading_1m_scaled,fixed_deposit_3m_scaled,recurring_deposit_3m_scaled,term_deposit_3m_scaled,provident_fund_3m_scaled,nps_3m_scaled,mutual_fund_3m_scaled,trading_3m_scaled,fixed_deposit_6m_scaled,recurring_deposit_6m_scaled,term_deposit_6m_scaled,provident_fund_6m_scaled,nps_6m_scaled,mutual_fund_6m_scaled,trading_6m_scaled,fixed_deposit_12m_scaled,recurring_deposit_12m_scaled,term_deposit_12m_scaled,provident_fund_12m_scaled,nps_12m_scaled,mutual_fund_12m_scaled,trading_12m_scaled,salary_1m,salary_3m,salary_6m,salary_12m,amt_insurance_spend_1m,amt_investment_spend_1m,amt_medical_spend_1m,amt_bills_utilities_spend_1m,amt_shopping_purchase_spend_1m,amt_online_shopping_spend_1m,amt_entertainment_spend_1m,amt_groceries_spend_1m,amt_education_spend_1m,amt_travel_spend_1m,amt_emi_repayment_spend_1m,amt_housing_services_spend_1m,amt_vehicle_expenses_spend_1m,amt_insurance_spend_3m,amt_investment_spend_3m,amt_medical_spend_3m,amt_bills_utilities_spend_3m,amt_shopping_purchase_spend_3m,amt_online_shopping_spend_3m,amt_entertainment_spend_3m,amt_groceries_spend_3m,amt_education_spend_3m,amt_travel_spend_3m,amt_emi_repayment_spend_3m,amt_housing_services_spend_3m,amt_vehicle_expenses_spend_3m,amt_insurance_spend_6m,amt_investment_spend_6m,amt_medical_spend_6m,amt_bills_utilities_spend_6m,amt_shopping_purchase_spend_6m,amt_online_shopping_spend_6m,amt_entertainment_spend_6m,amt_groceries_spend_6m,amt_education_spend_6m,amt_travel_spend_6m,amt_emi_repayment_spend_6m,amt_housing_services_spend_6m,amt_vehicle_expenses_spend_6m,amt_insurance_spend_12m,amt_investment_spend_12m,amt_medical_spend_12m,amt_bills_utilities_spend_12m,amt_shopping_purchase_spend_12m,amt_online_shopping_spend_12m,amt_entertainment_spend_12m,amt_groceries_spend_12m,amt_education_spend_12m,amt_travel_spend_12m,amt_emi_repayment_spend_12m,amt_housing_services_spend_12m,amt_vehicle_expenses_spend_12m
0,9359701533,2953643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8940337760,3961083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# anagog feature

In [38]:
sql = f"""
select
customerid,
IsGymLover,
IsSingle,
IsParent,
insuranceservices,

FROM `abcd-dataplatform-prod.abcd_anagog_transformed.abcd_anagog_data_transformed`
where flag=1
"""
angog_df = client.query(sql).to_dataframe()
angog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2441307 entries, 0 to 2441306
Data columns (total 5 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   customerid         object
 1   IsGymLover         object
 2   IsSingle           object
 3   IsParent           object
 4   insuranceservices  object
dtypes: object(5)
memory usage: 93.1+ MB


In [39]:
#angog_df[angog_df['insuranceservices'].str.contains('health', case=False, na=False)]
angog_df.customerid = angog_df.customerid.astype('int64')
angog_df.rename(columns={'customerid':'customer_id'},inplace=True)

In [40]:
angog_df['IsGymLover'] = angog_df['IsGymLover'].apply(lambda x: 1 if str(x).upper() == 'TRUE' else 0)
angog_df['IsSingle'] = angog_df['IsSingle'].apply(lambda x: 1 if str(x).upper() == 'TRUE' else 0)
angog_df['IsParent'] = angog_df['IsParent'].apply(lambda x: 1 if str(x).upper() == 'TRUE' else 0)
angog_df['insuranceservices'] = angog_df['insuranceservices'].apply(lambda x: 1 if str(x).upper() == 'TRUE' else 0)

In [41]:
angog_df.head()

,customer_id,IsGymLover,IsSingle,IsParent,insuranceservices
0,3609561,0,0,0,0
1,3666062,0,0,1,0
2,3635320,0,0,1,0
3,3615942,0,0,0,0
4,365505,0,0,0,0


In [42]:
angog_df.IsGymLover.value_counts()

IsGymLover
0    2432599
1       8708
Name: count, dtype: int64

In [43]:
# merging
model_base = model_base.merge(angog_df, on="customer_id", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078790 entries, 0 to 5078789
Columns: 146 entries, mobilenumber to insuranceservices
dtypes: Int64(1), float64(144), int64(1)
memory usage: 5.5 GB


In [44]:
model_base.head()

,mobilenumber,customer_id,fixed_deposit_1m,fixed_deposit_3m,fixed_deposit_6m,fixed_deposit_12m,recurring_deposit_1m,recurring_deposit_3m,recurring_deposit_6m,recurring_deposit_12m,term_deposit_1m,term_deposit_3m,term_deposit_6m,term_deposit_12m,provident_fund_1m,provident_fund_3m,provident_fund_6m,provident_fund_12m,nps_1m,nps_3m,nps_6m,nps_12m,mutual_fund_1m,mutual_fund_3m,mutual_fund_6m,mutual_fund_12m,trading_1m,trading_3m,trading_6m,trading_12m,fixed_deposit_1m_3m_ratio,fixed_deposit_3m_6m_ratio,fixed_deposit_6m_12m_ratio,fixed_deposit_3m_12m_ratio,recurring_deposit_1m_3m_ratio,recurring_deposit_3m_6m_ratio,recurring_deposit_6m_12m_ratio,recurring_deposit_3m_12m_ratio,term_deposit_1m_3m_ratio,term_deposit_3m_6m_ratio,term_deposit_6m_12m_ratio,term_deposit_3m_12m_ratio,provident_fund_1m_3m_ratio,provident_fund_3m_6m_ratio,provident_fund_6m_12m_ratio,provident_fund_3m_12m_ratio,nps_1m_3m_ratio,nps_3m_6m_ratio,nps_6m_12m_ratio,nps_3m_12m_ratio,mutual_fund_1m_3m_ratio,mutual_fund_3m_6m_ratio,mutual_fund_6m_12m_ratio,mutual_fund_3m_12m_ratio,trading_1m_3m_ratio,trading_3m_6m_ratio,trading_6m_12m_ratio,trading_3m_12m_ratio,fixed_deposit_1m_scaled,recurring_deposit_1m_scaled,term_deposit_1m_scaled,provident_fund_1m_scaled,nps_1m_scaled,mutual_fund_1m_scaled,trading_1m_scaled,fixed_deposit_3m_scaled,recurring_deposit_3m_scaled,term_deposit_3m_scaled,provident_fund_3m_scaled,nps_3m_scaled,mutual_fund_3m_scaled,trading_3m_scaled,fixed_deposit_6m_scaled,recurring_deposit_6m_scaled,term_deposit_6m_scaled,provident_fund_6m_scaled,nps_6m_scaled,mutual_fund_6m_scaled,trading_6m_scaled,fixed_deposit_12m_scaled,recurring_deposit_12m_scaled,term_deposit_12m_scaled,provident_fund_12m_scaled,nps_12m_scaled,mutual_fund_12m_scaled,trading_12m_scaled,salary_1m,salary_3m,salary_6m,salary_12m,amt_insurance_spend_1m,amt_investment_spend_1m,amt_medical_spend_1m,amt_bills_utilities_spend_1m,amt_shopping_purchase_spend_1m,amt_online_shopping_spend_1m,amt_entertainment_spend_1m,amt_groceries_spend_1m,amt_education_spend_1m,amt_travel_spend_1m,amt_emi_repayment_spend_1m,amt_housing_services_spend_1m,amt_vehicle_expenses_spend_1m,amt_insurance_spend_3m,amt_investment_spend_3m,amt_medical_spend_3m,amt_bills_utilities_spend_3m,amt_shopping_purchase_spend_3m,amt_online_shopping_spend_3m,amt_entertainment_spend_3m,amt_groceries_spend_3m,amt_education_spend_3m,amt_travel_spend_3m,amt_emi_repayment_spend_3m,amt_housing_services_spend_3m,amt_vehicle_expenses_spend_3m,amt_insurance_spend_6m,amt_investment_spend_6m,amt_medical_spend_6m,amt_bills_utilities_spend_6m,amt_shopping_purchase_spend_6m,amt_online_shopping_spend_6m,amt_entertainment_spend_6m,amt_groceries_spend_6m,amt_education_spend_6m,amt_travel_spend_6m,amt_emi_repayment_spend_6m,amt_housing_services_spend_6m,amt_vehicle_expenses_spend_6m,amt_insurance_spend_12m,amt_investment_spend_12m,amt_medical_spend_12m,amt_bills_utilities_spend_12m,amt_shopping_purchase_spend_12m,amt_online_shopping_spend_12m,amt_entertainment_spend_12m,amt_groceries_spend_12m,amt_education_spend_12m,amt_travel_spend_12m,amt_emi_repayment_spend_12m,amt_housing_services_spend_12m,amt_vehicle_expenses_spend_12m,IsGymLover,IsSingle,IsParent,insuranceservices
0,9359701533,2953643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,8940337760,3961083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [45]:
#target_base.to_csv('Invest_Sms_data_FD.csv',index=False)

In [46]:
%pwd

'/home/jupyter/Raish Azam/Investment Intent Model'

# bureau income feature

In [47]:
bureau_inc_query = f'''
    WITH base AS (
        SELECT 
            CUSTOMER_ID,
            MAX(income) AS CUST_INCOME,
            MAX(income_freq) AS INCOME_FREQUENCY,
            MAX(OCCUP_STATUS_CD) AS OCCUPATION
        FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_EMPLOYMENT_DAILY_BASE`
        GROUP BY CUSTOMER_ID
    )
    SELECT * FROM base
'''
client.query(bureau_inc_query).to_dataframe()

,CUSTOMER_ID,CUST_INCOME,INCOME_FREQUENCY,OCCUPATION
0,9731173649,150001,6,None
1,9696460413,50000,6,None
2,9971781174,17000,None,None
3,7567235644,10000,None,None
4,8058387927,15000,None,None
...,...,...,...,...
1951690,7799009343,50001,6,99
1951691,7548889924,50001,None,99
1951692,9388066111,50001,None,99
1951693,9337970229,50001,None,99


In [48]:
 bureau_inc_query = f'''
    with base as (
    select 
    "{as_of_end_dt}" as REF_MONTH,
    CUSTOMER_ID,
    max(income) as CUST_INCOME,
    max(income_freq) INCOME_FREQUENCY,
    max(OCCUP_STATUS_CD) as OCCUPATION,
    from `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_EMPLOYMENT_DAILY_BASE`
    WHERE CUSTOMER_ID IN (
        SELECT a.mobilenumber 
        FROM `abcd-dataplatform-prod.abcd_mobileapp_raw.ABCDPRODDB_t_customer` a
        WHERE DATE(createddate) <= DATE('{created_date_cutoff}')
    )
    group by 
    REF_MONTH,
    CUSTOMER_ID
    )

    select 
    REF_MONTH, CUSTOMER_ID,
    CASE WHEN OCCUPATION = '9' THEN "1"
         WHEN OCCUPATION = '10' THEN "2"
         WHEN OCCUPATION = '2' THEN "3"
         WHEN OCCUPATION = '99' THEN "4"
         ELSE null END as OCCUPATION,

      CASE 
        WHEN INCOME_FREQUENCY = '4' THEN CAST(CUST_INCOME as INT) * 3
        WHEN INCOME_FREQUENCY = '6' THEN CAST(CUST_INCOME as INT) * 2
        ELSE CAST(CUST_INCOME as INT)
      END AS Annual_Income

    from base
    '''
# Run the query
bureau_inc_df = client.query(bureau_inc_query).to_dataframe()
bureau_inc_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658363 entries, 0 to 1658362
Data columns (total 4 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   REF_MONTH      1658363 non-null  object
 1   CUSTOMER_ID    1658363 non-null  object
 2   OCCUPATION     1604991 non-null  object
 3   Annual_Income  717815 non-null   Int64 
dtypes: Int64(1), object(3)
memory usage: 330.4 MB


In [49]:
bureau_inc_df.OCCUPATION.value_counts()

OCCUPATION
4    1057891
1     358549
3     171697
2      16854
Name: count, dtype: int64

In [50]:
dictx = {
    1.0: 'Salaried',
    2.0: 'Self Employed',
    3.0: 'Self Employed Professional',
    4.0: 'Unknown'
}

# Convert OCCUPATION to numeric (in case it's still string)
bureau_inc_df['OCCUPATION'] = pd.to_numeric(bureau_inc_df['OCCUPATION'], errors='coerce')
bureau_inc_df

,REF_MONTH,CUSTOMER_ID,OCCUPATION,Annual_Income
0,2025-03-31 00:00:00,9600363485,NaN,100002
1,2025-03-31 00:00:00,9844586812,NaN,200002
2,2025-03-31 00:00:00,7208886812,NaN,8514
3,2025-03-31 00:00:00,9324433646,NaN,15000
4,2025-03-31 00:00:00,8058198203,NaN,16000
...,...,...,...,...
1658358,2025-03-31 00:00:00,8850732670,4.0,<NA>
1658359,2025-03-31 00:00:00,7439234184,4.0,25000
1658360,2025-03-31 00:00:00,9573989320,4.0,40000
1658361,2025-03-31 00:00:00,8145161924,4.0,39000


In [51]:
# Apply the mapping
bureau_inc_df['OCCUPATION'] = bureau_inc_df['OCCUPATION'].map(dictx)

In [52]:
bureau_inc_df.OCCUPATION.value_counts()

OCCUPATION
Unknown                       1057891
Salaried                       358549
Self Employed Professional     171697
Self Employed                   16854
Name: count, dtype: int64

In [53]:
bureau_inc_df.rename({'Annual_Income':'bureau_income','CUSTOMER_ID':'mobilenumber'},inplace=True,axis=1)

In [54]:
bureau_inc_df.drop('REF_MONTH',axis=1,inplace=True)

In [55]:
bureau_inc_df.rename(columns=lambda x : x.lower(),inplace=True)

In [56]:
bureau_inc_df.mobilenumber = bureau_inc_df['mobilenumber'].astype('int64')

In [57]:
bureau_inc_df.columns

Index(['mobilenumber', 'occupation', 'bureau_income'], dtype='object')

In [58]:
bureau_inc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658363 entries, 0 to 1658362
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   mobilenumber   1658363 non-null  int64 
 1   occupation     1604991 non-null  object
 2   bureau_income  717815 non-null   Int64 
dtypes: Int64(1), int64(1), object(1)
memory usage: 39.5+ MB


In [59]:
# merging
model_base = model_base.merge(bureau_inc_df, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078790 entries, 0 to 5078789
Columns: 148 entries, mobilenumber to bureau_income
dtypes: Int64(2), float64(144), int64(1), object(1)
memory usage: 5.6+ GB


# bureau dob feature

In [60]:
bureau_dob_query = f"""
SELECT 
    "{as_of_end_dt}" AS REF_MONTH,
    REGEXP_REPLACE(a.CUSTOMER_ID, r'[^0-9]', '') AS mobilenumber,
    a.DOB,
    a.GENDER
FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_NAME_DOB_DAILY_BASE` a
WHERE LENGTH(REGEXP_REPLACE(a.CUSTOMER_ID, r'[^0-9]', '')) = 10
"""

bureau_dob_df = client.query(bureau_dob_query).to_dataframe()
bureau_dob_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3680435 entries, 0 to 3680434
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   REF_MONTH     object
 1   mobilenumber  object
 2   DOB           object
 3   GENDER        object
dtypes: object(4)
memory usage: 937.1 MB


In [61]:
bureau_dob_df.head()

,REF_MONTH,mobilenumber,DOB,GENDER
0,2025-03-31 00:00:00,7830399820,01/01/1974,None
1,2025-03-31 00:00:00,8827141143,01/01/1976,None
2,2025-03-31 00:00:00,9179161499,01/01/1977,None
3,2025-03-31 00:00:00,8959482415,01/01/1980,None
4,2025-03-31 00:00:00,9113880046,01/01/1980,None


In [62]:
bureau_dob_df['mobilenumber'] = bureau_dob_df['mobilenumber'].astype('int64')

In [63]:
# Convert DOB and REF_MONTH to datetime
bureau_dob_df['DOB'] = pd.to_datetime(bureau_dob_df['DOB'], format='%d/%m/%Y')
bureau_dob_df['REF_MONTH'] = pd.to_datetime(bureau_dob_df['REF_MONTH'])
bureau_dob_df

,REF_MONTH,mobilenumber,DOB,GENDER
0,2025-03-31,7830399820,1974-01-01,None
1,2025-03-31,8827141143,1976-01-01,None
2,2025-03-31,9179161499,1977-01-01,None
3,2025-03-31,8959482415,1980-01-01,None
4,2025-03-31,9113880046,1980-01-01,None
...,...,...,...,...
3680430,2025-03-31,8428800641,2000-12-26,3
3680431,2025-03-31,7209001867,2004-06-27,3
3680432,2025-03-31,8521677875,1965-07-28,3
3680433,2025-03-31,9327306551,1980-12-31,3


In [64]:
# Calculate age in years
bureau_dob_df['age_in_years'] = (
    bureau_dob_df['REF_MONTH'].dt.year - bureau_dob_df['DOB'].dt.year
    - ((bureau_dob_df['REF_MONTH'].dt.month < bureau_dob_df['DOB'].dt.month) |
       ((bureau_dob_df['REF_MONTH'].dt.month == bureau_dob_df['DOB'].dt.month) &
        (bureau_dob_df['REF_MONTH'].dt.day < bureau_dob_df['DOB'].dt.day)))
)

bureau_dob_df

,REF_MONTH,mobilenumber,DOB,GENDER,age_in_years
0,2025-03-31,7830399820,1974-01-01,None,51
1,2025-03-31,8827141143,1976-01-01,None,49
2,2025-03-31,9179161499,1977-01-01,None,48
3,2025-03-31,8959482415,1980-01-01,None,45
4,2025-03-31,9113880046,1980-01-01,None,45
...,...,...,...,...,...
3680430,2025-03-31,8428800641,2000-12-26,3,24
3680431,2025-03-31,7209001867,2004-06-27,3,20
3680432,2025-03-31,8521677875,1965-07-28,3,59
3680433,2025-03-31,9327306551,1980-12-31,3,44


In [65]:
# Convert gender and occupation to numeric values
bureau_dob_df['GENDER'] = pd.to_numeric(bureau_dob_df['GENDER'], errors='coerce')
# Drop the DOB column
bureau_dob_df.drop('DOB', axis=1, inplace=True)

In [66]:
bureau_dob_df.GENDER.value_counts()

GENDER
2.0    2908723
1.0     657748
3.0       3205
4.0         65
Name: count, dtype: int64

In [67]:
# Map the occupation codes to meaningful labels
dict1 = {2.0: 'Male', 1.0: 'Female', 3.0: 'Others', 4.0: 'Unknown'}

In [68]:
# Apply the mapping
bureau_dob_df['GENDER'] = bureau_dob_df['GENDER'].map(dict1)

In [69]:
bureau_dob_df.drop('REF_MONTH',axis=1,inplace=True)

In [70]:
bureau_dob_df.rename(columns=lambda x : x.lower(),inplace=True)

In [71]:
# merging
model_base = model_base.merge(bureau_dob_df, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078794 entries, 0 to 5078793
Columns: 150 entries, mobilenumber to age_in_years
dtypes: Int64(2), float64(145), int64(1), object(2)
memory usage: 5.7+ GB


In [72]:
model_base.columns

Index(['mobilenumber', 'customer_id', 'fixed_deposit_1m', 'fixed_deposit_3m',
       'fixed_deposit_6m', 'fixed_deposit_12m', 'recurring_deposit_1m',
       'recurring_deposit_3m', 'recurring_deposit_6m', 'recurring_deposit_12m',
       ...
       'amt_housing_services_spend_12m', 'amt_vehicle_expenses_spend_12m',
       'IsGymLover', 'IsSingle', 'IsParent', 'insuranceservices', 'occupation',
       'bureau_income', 'gender', 'age_in_years'],
      dtype='object', length=150)

# Bureau INQ related features 

In [73]:
as_of_end_dt

Timestamp('2025-03-31 00:00:00')

# inq for pl

In [74]:
query = f"""
WITH parsed_data AS (
    SELECT 
        *, 
        PARSE_DATE('%d/%m/%Y', INQ_DATE) AS parsed_inq_date
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ENQ_DAILY_BASE`
    WHERE INQ_PURP_CD = '13' -- Personal Loan enquiry
),

filtered_enq AS (
    SELECT 
        CUSTOMER_ID,
        parsed_inq_date,
        SAFE_CAST(REPLACE(AMOUNT, ',', '') AS FLOAT64) AS AMOUNT_FLOAT
    FROM parsed_data
    WHERE parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 365 DAY)
                             AND DATE('{as_of_end_dt}')
),

aggregated AS (
    SELECT 
        CUSTOMER_ID,
        
        -- Last 3 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}')) AS pl_enq_count_3m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS pl_enq_amt_3m,
        
        -- Last 6 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}')) AS pl_enq_count_6m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS pl_enq_amt_6m
                 
        

    FROM filtered_enq
    GROUP BY CUSTOMER_ID
)

SELECT * FROM aggregated
"""
df_pl = client.query(query).to_dataframe()
df_pl.head()

,CUSTOMER_ID,pl_enq_count_3m,pl_enq_amt_3m,pl_enq_count_6m,pl_enq_amt_6m
0,8290348792,1,15999.0,3,115999.0
1,7568353484,1,2000000.0,3,5500000.0
2,8141124198,1,23999.0,1,23999.0
3,9356485527,1,2500.0,1,2500.0
4,9830723354,1,4500.0,2,54500.0


In [75]:
df_pl.CUSTOMER_ID.nunique(),df_pl.shape[0]

(794375, 794375)

In [76]:
df_pl.rename(columns={'CUSTOMER_ID':'mobilenumber'},inplace=True)
df_pl.mobilenumber = df_pl['mobilenumber'].astype('int64')
df_pl.rename(columns=lambda x : x.lower(),inplace=True)

In [77]:
# merging
model_base = model_base.merge(df_pl, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078794 entries, 0 to 5078793
Columns: 154 entries, mobilenumber to pl_enq_amt_6m
dtypes: Int64(4), float64(147), int64(1), object(2)
memory usage: 5.8+ GB


# inq for hfl

In [78]:
query = f"""
WITH parsed_data AS (
    SELECT 
        CUSTOMER_ID,
        PARSE_DATE('%d/%m/%Y', INQ_DATE) AS parsed_inq_date,
        SAFE_CAST(REPLACE(AMOUNT, ',', '') AS FLOAT64) AS AMOUNT_FLOAT
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ENQ_DAILY_BASE`
    WHERE INQ_PURP_CD = '14' -- Home Loan enquiry
),

filtered_enq AS (
    SELECT 
        CUSTOMER_ID,
        parsed_inq_date,
        AMOUNT_FLOAT
    FROM parsed_data
    WHERE parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 365 DAY)
                             AND DATE('{as_of_end_dt}')
),

aggregated AS (
    SELECT 
        CUSTOMER_ID,
        
        -- Last 3 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}')) AS hfl_enq_count_3m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS hfl_enq_amt_3m,
        
        -- Last 6 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}')) AS hfl_enq_count_6m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS hfl_enq_amt_6m

    FROM filtered_enq
    GROUP BY CUSTOMER_ID
)

SELECT * FROM aggregated
"""
df_hfl = client.query(query).to_dataframe()
df_hfl.head()


,CUSTOMER_ID,hfl_enq_count_3m,hfl_enq_amt_3m,hfl_enq_count_6m,hfl_enq_amt_6m
0,7725853055,1,0.0,1,0.0
1,8983366809,2,0.0,2,0.0
2,8097258312,2,0.0,2,0.0
3,6295939121,2,0.0,2,0.0
4,7889480334,1,0.0,1,0.0


In [79]:
df_hfl.CUSTOMER_ID.nunique(),df_hfl.shape[0]

(9631, 9631)

In [80]:
df_hfl.rename(columns={'CUSTOMER_ID':'mobilenumber'},inplace=True)
df_hfl.mobilenumber = df_hfl['mobilenumber'].astype('int64')
df_hfl.rename(columns=lambda x : x.lower(),inplace=True)

In [81]:
# merging
model_base = model_base.merge(df_hfl, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078794 entries, 0 to 5078793
Columns: 158 entries, mobilenumber to hfl_enq_amt_6m
dtypes: Int64(6), float64(149), int64(1), object(2)
memory usage: 6.0+ GB


In [82]:
query = f"""
WITH parsed_data AS (
    SELECT 
        CUSTOMER_ID,
        PARSE_DATE('%d/%m/%Y', INQ_DATE) AS parsed_inq_date,
        SAFE_CAST(REPLACE(AMOUNT, ',', '') AS FLOAT64) AS AMOUNT_FLOAT
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ENQ_DAILY_BASE`
    WHERE INQ_PURP_CD = '3' -- Business Loan enquiry
),

filtered_enq AS (
    SELECT 
        CUSTOMER_ID,
        parsed_inq_date,
        AMOUNT_FLOAT
    FROM parsed_data
    WHERE parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 365 DAY)
                             AND DATE('{as_of_end_dt}')
),

aggregated AS (
    SELECT 
        CUSTOMER_ID,
        
        -- Last 3 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}')) AS bl_enq_count_3m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS bl_enq_amt_3m,
        
        -- Last 6 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}')) AS bl_enq_count_6m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS bl_enq_amt_6m

    FROM filtered_enq
    GROUP BY CUSTOMER_ID
)

SELECT * FROM aggregated
"""
df_bl = client.query(query).to_dataframe()
df_bl.head()


,CUSTOMER_ID,bl_enq_count_3m,bl_enq_amt_3m,bl_enq_count_6m,bl_enq_amt_6m
0,9894366778,0,0.0,1,10001.0
1,9303140713,1,1300000.0,1,1300000.0
2,8899422666,0,0.0,1,1900000.0
3,9817203191,0,0.0,1,970000.0
4,9452306001,0,0.0,1,800000.0


In [83]:
df_bl.CUSTOMER_ID.nunique(),df_bl.shape[0]

(26459, 26459)

In [84]:
df_bl.rename(columns={'CUSTOMER_ID':'mobilenumber'},inplace=True)
df_bl.mobilenumber = df_bl['mobilenumber'].astype('int64')
df_bl.rename(columns=lambda x : x.lower(),inplace=True)

In [85]:
# merging
model_base = model_base.merge(df_bl, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078794 entries, 0 to 5078793
Columns: 162 entries, mobilenumber to bl_enq_amt_6m
dtypes: Int64(8), float64(151), int64(1), object(2)
memory usage: 6.2+ GB


In [86]:
query = f"""
WITH parsed_data AS (
    SELECT 
        CUSTOMER_ID,
        PARSE_DATE('%d/%m/%Y', INQ_DATE) AS parsed_inq_date,
        SAFE_CAST(REPLACE(AMOUNT, ',', '') AS FLOAT64) AS AMOUNT_FLOAT
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_ENQ_DAILY_BASE`
    WHERE INQ_PURP_CD = '7' -- Credit Card enquiry
),

filtered_enq AS (
    SELECT 
        CUSTOMER_ID,
        parsed_inq_date,
        AMOUNT_FLOAT
    FROM parsed_data
    WHERE parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 365 DAY)
                             AND DATE('{as_of_end_dt}')
),

aggregated AS (
    SELECT 
        CUSTOMER_ID,
        
        -- Last 3 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}')) AS cc_enq_count_3m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 3 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS cc_enq_amt_3m,
        
        -- Last 6 months
        COUNTIF(parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}')) AS cc_enq_count_6m,
        SUM(CASE WHEN parsed_inq_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 6 MONTH) AND DATE('{as_of_end_dt}') 
                 THEN AMOUNT_FLOAT ELSE 0 END) AS cc_enq_amt_6m

    FROM filtered_enq
    GROUP BY CUSTOMER_ID
)

SELECT * FROM aggregated
"""
df_cc = client.query(query).to_dataframe()
df_cc.head()


,CUSTOMER_ID,cc_enq_count_3m,cc_enq_amt_3m,cc_enq_count_6m,cc_enq_amt_6m
0,9316965328,1,10000.0,2,20000.0
1,7063196920,1,10000.0,1,10000.0
2,9633585431,1,10000.0,2,20000.0
3,6900324137,1,10000.0,1,10000.0
4,8743852915,2,200000.0,2,200000.0


In [87]:
df_cc.CUSTOMER_ID.nunique(),df_cc.shape[0]

(292721, 292721)

In [88]:
df_cc.rename(columns={'CUSTOMER_ID':'mobilenumber'},inplace=True)
df_cc.mobilenumber = df_cc['mobilenumber'].astype('int64')
df_cc.rename(columns=lambda x : x.lower(),inplace=True)

In [89]:
# merging
model_base = model_base.merge(df_cc, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078794 entries, 0 to 5078793
Columns: 166 entries, mobilenumber to cc_enq_amt_6m
dtypes: Int64(10), float64(153), int64(1), object(2)
memory usage: 6.3+ GB


# loan closed count

In [90]:
loan_close_count=f"""


WITH parsed_data AS (
    SELECT 
        CUSTOMER_ID,
        ACCT_TYPE_CD,
        PARSE_DATE('%d/%m/%Y', CLOSED_DT) AS closed_date
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
    WHERE ACCT_TYPE_CD IN ('123', '176', '195') -- Business loan, PL, HFL
),

aggregated AS (
    SELECT 
        CUSTOMER_ID,
        COUNTIF(ACCT_TYPE_CD = '123' AND closed_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 12 MONTH) AND DATE('{as_of_end_dt}')) AS pl_loan_closed_12m,
        COUNTIF(ACCT_TYPE_CD = '195' AND closed_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 12 MONTH) AND DATE('{as_of_end_dt}')) AS hfl_loan_closed_12m,
        COUNTIF(ACCT_TYPE_CD = '176'  AND closed_date BETWEEN DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 12 MONTH) AND DATE('{as_of_end_dt}')) AS business_loan_closed_12m
    FROM parsed_data
    GROUP BY CUSTOMER_ID
)

SELECT * FROM aggregated


"""

df_loan_count = client.query(loan_close_count).to_dataframe()
df_loan_count.head()

,CUSTOMER_ID,pl_loan_closed_12m,hfl_loan_closed_12m,business_loan_closed_12m
0,9880313448,49,0,0
1,9829604555,27,0,0
2,9491238489,25,0,0
3,6304356415,37,0,0
4,9766421657,22,0,0


In [91]:
df_loan_count.CUSTOMER_ID.nunique(),df_loan_count.shape[0]

(2656870, 2656870)

In [92]:
df_loan_count.rename(columns={'CUSTOMER_ID':'mobilenumber'},inplace=True)
df_loan_count.mobilenumber = df_loan_count['mobilenumber'].astype('int64')
df_loan_count.rename(columns=lambda x : x.lower(),inplace=True)

In [93]:
# merging
model_base = model_base.merge(df_loan_count, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078796 entries, 0 to 5078795
Columns: 169 entries, mobilenumber to business_loan_closed_12m
dtypes: Int64(13), float64(153), int64(1), object(2)
memory usage: 6.5+ GB


# pl_open_count_last 12month

In [94]:
loan_open_count=f"""

WITH parsed_data AS (
    SELECT 
        CUSTOMER_ID,
        ACCT_TYPE_CD,
        PARSE_DATE('%d/%m/%Y', OPEN_DT) AS open_date
    FROM `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
    WHERE ACCT_TYPE_CD = '123' -- PL
),

aggregated AS (
    SELECT 
        CUSTOMER_ID,
        COUNTIF(open_date >= DATE_SUB(DATE('{as_of_end_dt}'), INTERVAL 12 MONTH) AND open_date < DATE('{as_of_end_dt}')) AS pl_loan_open_count_12m
    FROM parsed_data
    GROUP BY CUSTOMER_ID
)

SELECT * FROM aggregated

"""

df_loan_open_count = client.query(loan_open_count).to_dataframe()
df_loan_open_count.head()

,CUSTOMER_ID,pl_loan_open_count_12m
0,9869111411,84
1,9995528502,28
2,7048355551,53
3,8886154325,43
4,8553138339,20


In [95]:
df_loan_open_count.rename(columns={'CUSTOMER_ID':'mobilenumber'},inplace=True)
df_loan_open_count.mobilenumber = df_loan_open_count['mobilenumber'].astype('int64')
df_loan_open_count.rename(columns=lambda x : x.lower(),inplace=True)

In [96]:
# merging
model_base = model_base.merge(df_loan_open_count, on="mobilenumber", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078800 entries, 0 to 5078799
Columns: 170 entries, mobilenumber to pl_loan_open_count_12m
dtypes: Int64(14), float64(153), int64(1), object(2)
memory usage: 6.5+ GB


In [97]:
cc_QUERY = f"""
WITH 
MaxScrubDate AS (
    SELECT 
        MAX(SCRUB_DATE) AS max_scrub_date
    FROM
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
),
FinalScrubDate AS (
    SELECT
        CASE 
            WHEN EXTRACT(DAY FROM max_scrub_date) > 26 THEN max_scrub_date 
            ELSE DATE_SUB(max_scrub_date, INTERVAL 1 MONTH) 
        END AS scrub_date
    FROM
        MaxScrubDate
),
LatestScrubData AS (
    SELECT 
        CUSTOMER_ID, 
        ACCT_TYPE_CD, 
        PARSE_DATE('%d/%m/%Y', OPEN_DT) AS OPEN_DT, 
        CAST(BALANCE_AM AS FLOAT64) AS BALANCE_AM, 
        PARSE_DATE('%d/%m/%Y', BALANCE_DT) AS BALANCE_DT, 
        PARSE_DATE('%d/%m/%Y', CLOSED_DT) AS CLOSED_DT, 
        CAST(CREDIT_LIMIT_AM AS FLOAT64) AS CREDIT_LIMIT_AM, 
        CAST(ORIG_LOAN_AM AS FLOAT64) AS ORIG_LOAN_AM
    FROM 
        `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
    WHERE 
        SCRUB_DATE = (SELECT scrub_date FROM FinalScrubDate)
        AND PARSE_DATE('%d/%m/%Y', OPEN_DT) > DATE '1900-01-01'
        AND PARSE_DATE('%d/%m/%Y', OPEN_DT) < DATE '{as_of_end_dt.date()}'
        AND ACCT_TYPE_CD = '214'
)

SELECT 
    CUSTOMER_ID,
    BALANCE_AM,
    CREDIT_LIMIT_AM,
    SAFE_DIVIDE(BALANCE_AM, CREDIT_LIMIT_AM) AS credit_util_ratio
FROM LatestScrubData
WHERE BALANCE_AM IS NOT NULL AND CREDIT_LIMIT_AM IS NOT NULL
"""

df_credit_util = client.query(cc_QUERY).to_dataframe()

In [98]:
df_credit_util

,CUSTOMER_ID,BALANCE_AM,CREDIT_LIMIT_AM,credit_util_ratio
0,9443049772,0.0,10000.0,0.000000
1,9425317062,0.0,800000.0,0.000000
2,8867427434,25911.0,75000.0,0.345480
3,7695958584,10167.0,25000.0,0.406680
4,9950477769,405503.0,400000.0,1.013758
5,9000467168,41942.0,50000.0,0.838840


In [99]:
df_credit_util.credit_util_ratio.value_counts()

credit_util_ratio
0.000000    2
0.345480    1
0.406680    1
1.013758    1
0.838840    1
Name: count, dtype: int64

In [100]:
df_credit_util.CUSTOMER_ID.nunique(),df_credit_util.shape[0]

(6, 6)

# demographic tier features

In [101]:
QUERY = f"""
WITH CTE_Demog AS (
    SELECT 
        A.id AS Customer_ID,
        A.mobilenumber AS Mobile_Number,
        COALESCE(K.employment_type, L.employment_type) AS Employment_Type,
        COALESCE(
            EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM COALESCE(A.dateofbirth, G.dob, H.dob)),
            B.customer_age,
            EXTRACT(YEAR FROM CURRENT_DATE()) - EXTRACT(YEAR FROM PARSE_DATE('%Y-%m-%d', C.date_of_birth)),
            E.age,
            F.customer_age,
            S.age
        ) AS Customer_Age,
        COALESCE(
            M.pincode, 
            CAST(N.pincode AS STRING), 
            O.pincode, 
            CAST(P.pincode AS STRING), 
            CAST(Q.pincode AS STRING), 
            CAST(R.pincode AS STRING)
        ) AS Residential_PIN_Code,
        A.createddate AS createdDate,
        ROW_NUMBER() OVER (PARTITION BY A.id ORDER BY A.id) AS rn
    FROM 
        `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer` AS A 
    LEFT JOIN `abcd-dataplatform-prod.abcd_curated_zone.fact_customer_health_detail` AS B ON A.id = B.customer_id
    LEFT JOIN `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer_vas_recon` AS C ON A.id = C.customer_id
    LEFT JOIN `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_dha_data` AS E ON A.id = E.customer_id
    LEFT JOIN `abcd-dataplatform-prod.abcd_curated_zone.dim_customer` AS F ON A.id = F.customer_id
    LEFT JOIN `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_homeloan_customer` AS G ON A.id = G.customer_id
    LEFT JOIN `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_hi_proposer_insured_members` AS H ON A.id = H.customer_id
    LEFT JOIN (
        SELECT 
            customer_id,
            employment_type,
            SUM((monthly_income + 
                IFNULL(other_income_amount, 0) +
                IFNULL(other_income_amount_2, 0) +
                IFNULL(other_income_amount_3, 0) +
                IFNULL(other_income_amount_4, 0) +
                IFNULL(other_income_amount_5, 0) +
                IFNULL(other_income_amount_6, 0) +
                IFNULL(other_income_amount_7, 0)) * 12
            ) AS Annual_income
        FROM `abcd-dataplatform-prod.abcd_curated_zone.fact_homeloan_employement_detail`
        GROUP BY customer_id, employment_type
    ) AS K ON A.id = K.customer_id
    LEFT JOIN (
        SELECT 
            customer_id,
            employment_type,
            SUM((monthly_income + 
                IFNULL(other_income_amount, 0) +
                IFNULL(other_income_amount2, 0) +
                IFNULL(other_income_amount3, 0) +
                IFNULL(other_income_amount4, 0) +
                IFNULL(other_income_amount5, 0) +
                IFNULL(other_income_amount6, 0) +
                IFNULL(other_income_amount7, 0)) * 12
            ) AS Annual_income
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_homeloan_employment_dtl`
        GROUP BY customer_id, employment_type
    ) AS L ON A.id = L.customer_id AND K.employment_type = L.employment_type
    LEFT JOIN (
        SELECT customerid, pincode
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_address`
    ) AS M ON A.id = M.customerid
    LEFT JOIN (
        SELECT customer_id, MAX(pincode) AS pincode
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_homeloan_address`
        GROUP BY customer_id
    ) AS N ON A.id = N.customer_id
    LEFT JOIN (
        SELECT customer_id, MAX(pincode) AS pincode
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_pl_address`
        GROUP BY customer_id
    ) AS O ON A.id = O.customer_id
    LEFT JOIN (
        SELECT customer_id, MAX(pin_code) AS pincode
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_bl_businessloan_details`
        GROUP BY customer_id
    ) AS P ON A.id = CAST(P.customer_id AS INTEGER)
    LEFT JOIN (
        SELECT customer_id, MAX(pincode) AS pincode
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_mf_address_details`
        GROUP BY customer_id
    ) AS Q ON A.id = Q.customer_id
    LEFT JOIN (
        SELECT customer_id, MAX(kyc_pincode) AS pincode
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_li_address`
        GROUP BY customer_id
    ) AS R ON A.id = R.customer_id
    LEFT JOIN (
        SELECT customer_id, MAX(age) AS age
        FROM `abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_li_quote_details`
        GROUP BY customer_id
    ) AS S ON A.id = S.customer_id
)

SELECT 
    Customer_ID,
    Mobile_Number,
    Employment_Type,
    Customer_Age,
    Residential_PIN_Code
FROM CTE_Demog
WHERE rn = 1 AND createdDate <= '{as_of_end_dt}';
"""

query_job = client.query(QUERY)
df_demogs = query_job.to_dataframe()
# Standardize column names to lowercase
df_demogs.rename(columns=lambda col: col.lower(), inplace=True)

In [102]:
df_demogs.head()

,customer_id,mobile_number,employment_type,customer_age,residential_pin_code
0,43673,9667526749,None,<NA>,None
1,44440,9723328920,None,45,None
2,64611,9829074008,None,37,None
3,68427,9030731124,None,<NA>,None
4,71141,8249185565,None,42,None


In [103]:
appsflyer_query = """
    WITH Appsflyer_records AS (
        SELECT
            COALESCE(CAST(mp_user_id AS INT64), CAST(guestid AS INT64)) AS customer_id,
            B.postal_code AS PINCODE_APPSFLYER,
            ROW_NUMBER() OVER (PARTITION BY COALESCE(CAST(mp_user_id AS INT64), CAST(guestid AS INT64)) ORDER BY A.time DESC) AS row_num
        FROM abcd-dataplatform-prod.abcd_mixpanel_raw.abcd_mp_master_event A
        LEFT JOIN abcd-dataplatform-prod.abcd_curated_zone.fact_appsflyer_metrics B
        ON A.appsflyerid = B.appsflyer_id
    )
    SELECT DISTINCT customer_id, PINCODE_APPSFLYER
    FROM Appsflyer_records
    WHERE row_num = 1 AND customer_id != 0
"""
df_appsflyer = client.query(appsflyer_query).to_dataframe()
df_appsflyer.drop_duplicates(subset=['customer_id'], inplace=True)

In [104]:
df_appsflyer.head()

,customer_id,PINCODE_APPSFLYER
0,102995,831004
1,1019157,140603
2,1034087,110041
3,1164543,201003
4,1372944,695103


In [105]:
# Step 3: Merge with demographics to ensure consistent customer_id list
df_merged = pd.merge(df_demogs, df_appsflyer, on='customer_id', how='left')

# Step 4: Final pincode fallback
df_merged['final_pincode'] = df_merged['PINCODE_APPSFLYER'].fillna('')
df_merged['final_pincode'] = df_merged['final_pincode'].astype(str).str[:6]

In [106]:
# Step 5: Pincode Mapper
pincode_mapper_query = """
    SELECT pincode, Tier
    FROM abcd-dataplatform.abcd_data_science_app.PINCODE_MAPPER
"""
pincode_mapper = client.query(pincode_mapper_query).to_dataframe()
pincode_mapper['pincode'] = pincode_mapper['pincode'].astype(str).str[:6]

In [107]:
# Step 6: Merge Tier with demographics
df_tier = pd.merge(df_merged[['customer_id', 'customer_age', 'final_pincode']], pincode_mapper, left_on='final_pincode', right_on='pincode', how='left')
df_tier = df_tier[['customer_id', 'Tier']]
df_tier

,customer_id,Tier
0,43673,NaN
1,44440,Tier2
2,64611,Tier3
3,68427,NaN
4,71141,Tier2
...,...,...
5059006,1240068,NaN
5059007,1959239,NaN
5059008,3545362,NaN
5059009,3907493,NaN


In [108]:
df_tier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5059011 entries, 0 to 5059010
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   customer_id  Int64 
 1   Tier         object
dtypes: Int64(1), object(1)
memory usage: 82.0+ MB


In [109]:
# merging
model_base = model_base.merge(df_tier, on="customer_id", how="left")
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078800 entries, 0 to 5078799
Columns: 171 entries, mobilenumber to Tier
dtypes: Int64(14), float64(153), int64(1), object(3)
memory usage: 6.5+ GB


In [110]:
model_base.head()

,mobilenumber,customer_id,fixed_deposit_1m,fixed_deposit_3m,fixed_deposit_6m,fixed_deposit_12m,recurring_deposit_1m,recurring_deposit_3m,recurring_deposit_6m,recurring_deposit_12m,term_deposit_1m,term_deposit_3m,term_deposit_6m,term_deposit_12m,provident_fund_1m,provident_fund_3m,provident_fund_6m,provident_fund_12m,nps_1m,nps_3m,nps_6m,nps_12m,mutual_fund_1m,mutual_fund_3m,mutual_fund_6m,mutual_fund_12m,trading_1m,trading_3m,trading_6m,trading_12m,fixed_deposit_1m_3m_ratio,fixed_deposit_3m_6m_ratio,fixed_deposit_6m_12m_ratio,fixed_deposit_3m_12m_ratio,recurring_deposit_1m_3m_ratio,recurring_deposit_3m_6m_ratio,recurring_deposit_6m_12m_ratio,recurring_deposit_3m_12m_ratio,term_deposit_1m_3m_ratio,term_deposit_3m_6m_ratio,term_deposit_6m_12m_ratio,term_deposit_3m_12m_ratio,provident_fund_1m_3m_ratio,provident_fund_3m_6m_ratio,provident_fund_6m_12m_ratio,provident_fund_3m_12m_ratio,nps_1m_3m_ratio,nps_3m_6m_ratio,nps_6m_12m_ratio,nps_3m_12m_ratio,mutual_fund_1m_3m_ratio,mutual_fund_3m_6m_ratio,mutual_fund_6m_12m_ratio,mutual_fund_3m_12m_ratio,trading_1m_3m_ratio,trading_3m_6m_ratio,trading_6m_12m_ratio,trading_3m_12m_ratio,fixed_deposit_1m_scaled,recurring_deposit_1m_scaled,term_deposit_1m_scaled,provident_fund_1m_scaled,nps_1m_scaled,mutual_fund_1m_scaled,trading_1m_scaled,fixed_deposit_3m_scaled,recurring_deposit_3m_scaled,term_deposit_3m_scaled,provident_fund_3m_scaled,nps_3m_scaled,mutual_fund_3m_scaled,trading_3m_scaled,fixed_deposit_6m_scaled,recurring_deposit_6m_scaled,term_deposit_6m_scaled,provident_fund_6m_scaled,nps_6m_scaled,mutual_fund_6m_scaled,trading_6m_scaled,fixed_deposit_12m_scaled,recurring_deposit_12m_scaled,term_deposit_12m_scaled,provident_fund_12m_scaled,nps_12m_scaled,mutual_fund_12m_scaled,trading_12m_scaled,salary_1m,salary_3m,salary_6m,salary_12m,amt_insurance_spend_1m,amt_investment_spend_1m,amt_medical_spend_1m,amt_bills_utilities_spend_1m,amt_shopping_purchase_spend_1m,amt_online_shopping_spend_1m,amt_entertainment_spend_1m,amt_groceries_spend_1m,amt_education_spend_1m,amt_travel_spend_1m,amt_emi_repayment_spend_1m,amt_housing_services_spend_1m,amt_vehicle_expenses_spend_1m,amt_insurance_spend_3m,amt_investment_spend_3m,amt_medical_spend_3m,amt_bills_utilities_spend_3m,amt_shopping_purchase_spend_3m,amt_online_shopping_spend_3m,amt_entertainment_spend_3m,amt_groceries_spend_3m,amt_education_spend_3m,amt_travel_spend_3m,amt_emi_repayment_spend_3m,amt_housing_services_spend_3m,amt_vehicle_expenses_spend_3m,amt_insurance_spend_6m,amt_investment_spend_6m,amt_medical_spend_6m,amt_bills_utilities_spend_6m,amt_shopping_purchase_spend_6m,amt_online_shopping_spend_6m,amt_entertainment_spend_6m,amt_groceries_spend_6m,amt_education_spend_6m,amt_travel_spend_6m,amt_emi_repayment_spend_6m,amt_housing_services_spend_6m,amt_vehicle_expenses_spend_6m,amt_insurance_spend_12m,amt_investment_spend_12m,amt_medical_spend_12m,amt_bills_utilities_spend_12m,amt_shopping_purchase_spend_12m,amt_online_shopping_spend_12m,amt_entertainment_spend_12m,amt_groceries_spend_12m,amt_education_spend_12m,amt_travel_spend_12m,amt_emi_repayment_spend_12m,amt_housing_services_spend_12m,amt_vehicle_expenses_spend_12m,IsGymLover,IsSingle,IsParent,insuranceservices,occupation,bureau_income,gender,age_in_years,pl_enq_count_3m,pl_enq_amt_3m,pl_enq_count_6m,pl_enq_amt_6m,hfl_enq_count_3m,hfl_enq_amt_3m,hfl_enq_count_6m,hfl_enq_amt_6m,bl_enq_count_3m,bl_enq_amt_3m,bl_enq_count_6m,bl_enq_amt_6m,cc_enq_count_3m,cc_enq_amt_3m,cc_enq_count_6m,cc_enq_amt_6m,pl_loan_closed_12m,hfl_loan_closed_12m,business_loan_closed_12m,pl_loan_open_count_12m,Tier
0,9359701533,2953643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [111]:
QUERY = f""" 
SELECT
    DISTINCT 
    SAFE_CAST(CUST_ID AS INT64) AS mobilenumber,
    SAFE_CAST(Score_V3 AS INT64) AS EXPERIAN_Score_V3
FROM
    abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_SCORE_DAILY_BASE
WHERE 
    SAFE_CAST(CUST_ID AS INT64) IS NOT NULL
    AND SAFE_CAST(Score_V3 AS INT64) IS NOT NULL
"""

#Saving them into DataFrame object
exp_cs_df = client.query(QUERY).to_dataframe()
exp_cs_df.mobilenumber = pd.to_numeric(exp_cs_df.mobilenumber, errors='coerce')
exp_cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3554219 entries, 0 to 3554218
Data columns (total 2 columns):
 #   Column             Dtype
---  ------             -----
 0   mobilenumber       Int64
 1   EXPERIAN_Score_V3  Int64
dtypes: Int64(2)
memory usage: 61.0 MB


In [112]:
model_base = model_base.merge(exp_cs_df,on='mobilenumber',how='left')
model_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078802 entries, 0 to 5078801
Columns: 172 entries, mobilenumber to EXPERIAN_Score_V3
dtypes: Int64(15), float64(153), int64(1), object(3)
memory usage: 6.6+ GB


In [113]:
fill_rate = model_base.notnull().mean().sort_values(ascending=False) * 100 #target_base.notnull().mean() non-null values ka proportion
fill_rate_df = fill_rate.reset_index()
fill_rate_df.columns = ['feature', 'fill_rate_percent']

In [114]:
fill_rate_df

,feature,fill_rate_percent
0,mobilenumber,100.000000
1,customer_id,100.000000
2,amt_vehicle_expenses_spend_3m,74.772633
3,amt_housing_services_spend_3m,74.772633
4,amt_emi_repayment_spend_3m,74.772633
5,amt_travel_spend_3m,74.772633
6,amt_education_spend_3m,74.772633
7,amt_groceries_spend_3m,74.772633
8,amt_entertainment_spend_3m,74.772633
9,amt_online_shopping_spend_3m,74.772633


In [115]:
import os

In [116]:
if not os.path.exists('snapshots'):
    os.makedirs('snapshots',exist_ok=True)

In [117]:
model_base.to_csv(f'snapshots/digigold_feature_{target_month.date()}.csv',index=False)

In [118]:
.

SyntaxError: invalid syntax (1933637684.py, line 1)

In [ ]:
target_base_query = f"""
     SELECT DISTINCT
        mobilenumber,
        month_year,
        type_of_investment,
        total_investment
    FROM 
        `abcd-dataplatform.abcd_data_science_app.INVESTMENT_MONTH_ON_MONTH_DATA`
    """
df = client.query(target_base_query).to_dataframe()
df.info()

In [ ]:
df.type_of_investment.unique()

In [ ]:
df[df['type_of_investment']=='DIGIGOLD']

In [ ]:
9866556741

In [ ]:
df[df['mobilenumber']==9866556741]